### 데이터 처리
N 사이즈 / ngram_syscall / exploit 개수

In [2]:
import pickle

## Load pickle
with open("/opt/output/perm/analysis/ngram_result.pkl","rb") as f:
    ngram_result = pickle.load(f)

In [3]:
import pandas as pd
# 초기화
ngramdf = pd.DataFrame(columns=["N", "Ngram_syscall", "num_of_exploit"])
item = dict()
for i in range(len(ngram_result)):
    for ngram_syscall, cnt in ngram_result[i].items():
        item["N"] = i+2
        item["Ngram_syscall"] = ngram_syscall.split(",")
        item["num_of_exploit"] = cnt
        ngramdf = ngramdf.append(item, ignore_index=True)
        item = dict()
ngramdf_origin = ngramdf.copy()
ngramdf_origin

,N,Ngram_syscall,num_of_exploit
0,2,"[12, 41]",3
1,2,"[257, 5]",40
2,2,"[107, 117]",3
3,2,"[3, 103]",3
4,2,"[8, 5]",4
...,...,...,...
11350,63,"[248, 248, 41, 248, 248, 248, 54, 0, 3, 5, 9, ...",2
11351,63,"[248, 248, 248, 41, 248, 248, 248, 54, 0, 3, 5...",2
11352,64,"[14, 248, 248, 248, 41, 248, 248, 248, 54, 0, ...",2
11353,64,"[248, 248, 248, 41, 248, 248, 248, 54, 0, 3, 5...",2


In [4]:
# 모든 익스플로잇 path laod
import subprocess
import json

pathFile = subprocess.check_output(f"find /opt/output/perm/path/*.json", shell=True).decode().strip().split()
EIDList = list(map(lambda x: x.replace("/opt/output/perm/path/","").replace(".json",""), pathFile))
pathList = dict() # 익스플로잇 코드 path dictionary의 dictionary - pathList[EID][path] = path 파일 정보
for EID in EIDList:
    with open(f"/opt/output/perm/path/{EID}.json", "r") as f:
        pathList[EID] = dict()
        pathList[EID]["path"] = json.load(f)

In [5]:
# dict {EID: all path list} 만들어 주기
eid_all_path_dict = dict()
for EID, pathset in pathList.items():
    eid_all_path_dict[EID] = list()
    for userfunc, pathset in pathset['path'].items():
        if not len(pathset) == 0:
            eid_all_path_dict[EID].extend(pathset)

In [6]:
import numpy as np
def find_EID_with_ngram(ngram, ngram_cnt):
    EID_with_ngram  = list()
    for EID, pathset in eid_all_path_dict.items():
        if ngram_cnt == len(EID_with_ngram):
            return EID_with_ngram
        for path in pathset:
            strpath = "," + ",".join(path) + ","
            if ngram in strpath:
                EID_with_ngram.append(EID)
                break
    if not EID_with_ngram: # if no EID
        return np.nan
    else:
        return EID_with_ngram

In [7]:
# ngrma이 나오는 EID 저장
import numpy as np
ngramdf["EIDs"] = np.nan
for index, row in ngramdf.iterrows():
    strngram = "," + ",".join(row.Ngram_syscall) + ","
    ngramdf["EIDs"][index] = find_EID_with_ngram(strngram, row.num_of_exploit)
ngramdf

/tmp/ipykernel_1611693/725562346.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ngramdf["EIDs"][index] = find_EID_with_ngram(strngram, row.num_of_exploit)
/home/mini/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,N,Ngram_syscall,num_of_exploit,EIDs
0,2,"[12, 41]",3,"[3, 50541, 926]"
1,2,"[257, 5]",40,"[1397, 14814, 15023, 15024, 15285, 15704, 1577..."
2,2,"[107, 117]",3,"[1397, 40871, 47170]"
3,2,"[3, 103]",3,"[47168, 47169, 47170]"
4,2,"[8, 5]",4,"[12, 3, 43127, 50541]"
...,...,...,...,...
11350,63,"[248, 248, 41, 248, 248, 248, 54, 0, 3, 5, 9, ...",2,NaN
11351,63,"[248, 248, 248, 41, 248, 248, 248, 54, 0, 3, 5...",2,NaN
11352,64,"[14, 248, 248, 248, 41, 248, 248, 248, 54, 0, ...",2,NaN
11353,64,"[248, 248, 248, 41, 248, 248, 248, 54, 0, 3, 5...",2,NaN


In [8]:
# N-gram 처리 (시스템콜 아닌 것들 바꿔주기, 유저함수 삭제)
import re

for index, row in ngramdf.iterrows():
    if "setns" in row.Ngram_syscall:
        row.Ngram_syscall = ",".join(row.Ngram_syscall).replace("setns","308").split(",")
    if "time" in row.Ngram_syscall:
        row.Ngram_syscall = ",".join(row.Ngram_syscall).replace("time","201").split(",")
    if "capset" in row.Ngram_syscall:
        row.Ngram_syscall = ",".join(row.Ngram_syscall).replace("capset","126").split(",")
    if "fstat" in row.Ngram_syscall:
        row.Ngram_syscall = ",".join(row.Ngram_syscall).replace("fstat","5").split(",")
    if "uname" in row.Ngram_syscall:
        row.Ngram_syscall = ",".join(row.Ngram_syscall).replace("uname","63").split(",")
    if "__NR_vm86old" in row.Ngram_syscall:
        row.Ngram_syscall = ",".join(row.Ngram_syscall).replace("__NR_vm86old","436").split(",")
    if "__NR_socketcall" in row.Ngram_syscall:
        row.Ngram_syscall = ",".join(row.Ngram_syscall).replace("__NR_socketcall","437").split(",")
    if "sigaction" in row.Ngram_syscall:
        row.Ngram_syscall = ",".join(row.Ngram_syscall).replace("sigaction","13").split(",")
    if "ioctl" in row.Ngram_syscall:
        row.Ngram_syscall = ",".join(row.Ngram_syscall).replace("ioctl","16").split(",")
    if "kmalloc" in row.Ngram_syscall:
        row.Ngram_syscall = ",".join(row.Ngram_syscall).replace("kmalloc","12").split(",")
    if "shutdown" in row.Ngram_syscall:
        row.Ngram_syscall = ",".join(row.Ngram_syscall).replace("shutdown","48").split(",")
    if "msgrcv" in row.Ngram_syscall:
        row.Ngram_syscall = ",".join(row.Ngram_syscall).replace("msgrcv","70").split(",")
    ngramdf.iloc[index]['Ngram_syscall'] = re.findall(r'\d+', ",".join(row.Ngram_syscall)).copy()
ngramdf

,N,Ngram_syscall,num_of_exploit,EIDs
0,2,"[12, 41]",3,"[3, 50541, 926]"
1,2,"[257, 5]",40,"[1397, 14814, 15023, 15024, 15285, 15704, 1577..."
2,2,"[107, 117]",3,"[1397, 40871, 47170]"
3,2,"[3, 103]",3,"[47168, 47169, 47170]"
4,2,"[8, 5]",4,"[12, 3, 43127, 50541]"
...,...,...,...,...
11350,63,"[248, 248, 41, 248, 248, 248, 54, 0, 3, 5, 9, ...",2,NaN
11351,63,"[248, 248, 248, 41, 248, 248, 248, 54, 0, 3, 5...",2,NaN
11352,64,"[14, 248, 248, 248, 41, 248, 248, 248, 54, 0, ...",2,NaN
11353,64,"[248, 248, 248, 41, 248, 248, 248, 54, 0, 3, 5...",2,NaN


In [9]:
import numpy as np
# Ngrma 처리 후 변경 사항 처리
for index, row in ngramdf.iterrows():
    # Ngram_syscall 없는 애들은 NaN로 바꿔주기
    if not row.Ngram_syscall:
        ngramdf.iloc[index]['N'] = np.nan
    elif not isinstance(row.Ngram_syscall, list):
        ngramdf.iloc[index]['N'] = np.nan
    elif len(row.Ngram_syscall) == row.N:      
        ngramdf.iloc[index]['N'] = len(row.Ngram_syscall)

ngramdf = ngramdf.dropna(axis=0)
ngramdf

,N,Ngram_syscall,num_of_exploit,EIDs
0,2,"[12, 41]",3,"[3, 50541, 926]"
1,2,"[257, 5]",40,"[1397, 14814, 15023, 15024, 15285, 15704, 1577..."
2,2,"[107, 117]",3,"[1397, 40871, 47170]"
3,2,"[3, 103]",3,"[47168, 47169, 47170]"
4,2,"[8, 5]",4,"[12, 3, 43127, 50541]"
...,...,...,...,...
11329,59,"[248, 248, 248, 54, 0, 3, 5, 9, 10, 12, 13, 14...",2,"[40871, 47170]"
11332,59,"[41, 248, 248, 248, 54, 0, 3, 5, 9, 10, 12, 13...",2,"[40871, 47170]"
11335,60,"[248, 248, 248, 54, 0, 3, 5, 9, 10, 12, 13, 14...",2,"[40871, 47170]"
11336,60,"[41, 248, 248, 248, 54, 0, 3, 5, 9, 10, 12, 13...",2,"[40871, 47170]"


### 시스템콜 번호 -> 이름 전환 함수

In [10]:
import subprocess
# 시스템콜 번호를 시스템콜 이름으로 바꾸기 위함 함수
def convert_num_name(sysnum):
    if sysnum == 436:
        return "vm86old"
    if sysnum == 437:
        return "socketcall"
    if sysnum == "clone":
        return "clone"
    if sysnum == "write":
        return "write"
    cmd = f'grep " {sysnum}$" /tmp/x86_64.syscall'
    sys_name = subprocess.check_output(cmd, shell=True).decode().strip().split()[1].replace("__NR_","")
    return sys_name

# Soft 중복 제거  N-gram

N-gram Syscall Sequence Pattern과 EIDs가 모두 같은 중복만 제거

In [11]:
# 중복제거를 위한 df 만들고 N이 큰순으로 정렬
soft_dedup_ngramdf = ngramdf.sort_values(by=["N"], ascending=False, ignore_index=True)
soft_dedup_ngramdf

,N,Ngram_syscall,num_of_exploit,EIDs
0,61,"[41, 248, 248, 248, 54, 0, 3, 5, 9, 10, 12, 13...",2,"[40871, 47170]"
1,60,"[41, 248, 248, 248, 54, 0, 3, 5, 9, 10, 12, 13...",2,"[40871, 47170]"
2,60,"[248, 248, 248, 54, 0, 3, 5, 9, 10, 12, 13, 14...",2,"[40871, 47170]"
3,59,"[41, 248, 248, 248, 54, 0, 3, 5, 9, 10, 12, 13...",2,"[40871, 47170]"
4,59,"[248, 248, 248, 54, 0, 3, 5, 9, 10, 12, 13, 14...",2,"[40871, 47170]"
...,...,...,...,...
9254,2,"[3, 230]",5,"[31574, 39230, 40616, 43345]"
9255,2,"[11, 3]",7,"[15944, 18411, 35161, 42275, 47163, 50541, 6851]"
9256,2,"[302, 230]",4,"[40871, 41995, 43127, 47170]"
9257,2,"[39, 257]",3,"[22362, 47163, 50541]"


In [12]:
import numpy as np

for index1, row1 in soft_dedup_ngramdf.iterrows():
    if row1.Ngram_syscall != row1.Ngram_syscall:# NaN이면 다음으로
        continue
    for index2, row2 in soft_dedup_ngramdf.loc[index1+1:].query(f'N != {row1.N}').iterrows():
        if row2.Ngram_syscall != row2.Ngram_syscall:# NaN이면 다음으로
            continue
        row1_seq = "," + ",".join(row1.Ngram_syscall) + ","
        row2_seq = "," + ",".join(row2.Ngram_syscall) + ","
        if row2_seq in row1_seq:
            if row1.EIDs == row2.EIDs:
                soft_dedup_ngramdf["Ngram_syscall"][index2] = np.nan    
soft_dedup_ngramdf = soft_dedup_ngramdf.dropna(axis=0)
soft_dedup_ngramdf

,N,Ngram_syscall,num_of_exploit,EIDs
0,61,"[41, 248, 248, 248, 54, 0, 3, 5, 9, 10, 12, 13...",2,"[40871, 47170]"
87,49,"[230, 0, 3, 5, 9, 10, 12, 13, 14, 17, 56, 218,...",2,"[40871, 47170]"
151,45,"[12, 12, 5, 0, 257, 4, 12, 12, 5, 0, 257, 4, 1...",2,"[47163, 50541]"
205,43,"[9, 0, 3, 5, 9, 10, 12, 13, 14, 17, 56, 218, 2...",2,"[40616, 43127]"
223,43,"[12, 12, 5, 0, 257, 4, 12, 12, 5, 0, 257, 4, 1...",2,"[47163, 50541]"
...,...,...,...,...
9251,2,"[9, 10]",12,"[33321, 33516, 35370, 40616, 40839, 40871, 419..."
9254,2,"[3, 230]",5,"[31574, 39230, 40616, 43345]"
9255,2,"[11, 3]",7,"[15944, 18411, 35161, 42275, 47163, 50541, 6851]"
9257,2,"[39, 257]",3,"[22362, 47163, 50541]"


In [13]:
# N 다시 조정
for index, row in soft_dedup_ngramdf.iterrows():
    if not len(row.Ngram_syscall) == row.N:
        soft_dedup_ngramdf["N"][index] = len(row.Ngram_syscall)
# 다시 정렬
soft_dedup_ngramdf = soft_dedup_ngramdf.sort_values(by=["N"], ascending=False, ignore_index=True)

In [14]:
# N=1인 것 제거
for index, row in soft_dedup_ngramdf.iterrows():
    if len(row.Ngram_syscall) == 1:
        soft_dedup_ngramdf["N"][index] = np.nan
soft_dedup_ngramdf = soft_dedup_ngramdf.dropna(axis=0)
soft_dedup_ngramdf

,N,Ngram_syscall,num_of_exploit,EIDs
0,61,"[41, 248, 248, 248, 54, 0, 3, 5, 9, 10, 12, 13...",2,"[40871, 47170]"
1,49,"[230, 0, 3, 5, 9, 10, 12, 13, 14, 17, 56, 218,...",2,"[40871, 47170]"
2,45,"[12, 12, 5, 0, 257, 4, 12, 12, 5, 0, 257, 4, 1...",2,"[47163, 50541]"
3,43,"[9, 0, 3, 5, 9, 10, 12, 13, 14, 17, 56, 218, 2...",2,"[40616, 43127]"
4,43,"[12, 12, 5, 0, 257, 4, 12, 12, 5, 0, 257, 4, 1...",2,"[47163, 50541]"
...,...,...,...,...
473,2,"[257, 0]",16,"[15024, 15774, 3, 35370, 40616, 40871, 42275, ..."
474,2,"[8, 0]",8,"[12, 20979, 27297, 3, 43127, 45553, 50808, 9083]"
475,2,"[3, 59]",9,"[39230, 40871, 43418, 47170, 6851, 9479, 9542]"
476,2,"[8, 5]",4,"[12, 3, 43127, 50541]"


In [15]:
# pickle로 저장
import pickle

with open("soft_dedup_ngramdf.pkl","wb") as f:
    pickle.dump(soft_dedup_ngramdf, f)

# Soft 불러오기

In [16]:
import pickle

## Load pickle
with open("soft_dedup_ngramdf.pkl","rb") as f:
    soft_dedup_ngramdf = pickle.load(f)

# 전체 익스플로잇에서 가장 많이 나온 N-gram


In [17]:
soft_dedup_ngramdf.sort_values(by=["num_of_exploit"], ascending=False).head(10)

,N,Ngram_syscall,num_of_exploit,EIDs
459,2,"[12, 12]",55,"[10613, 12, 1397, 14814, 15023, 15024, 15285, ..."
455,2,"[257, 5]",40,"[1397, 14814, 15023, 15024, 15285, 15704, 1577..."
399,2,"[0, 3]",38,"[1397, 14814, 15023, 15024, 15285, 15704, 1577..."
338,2,"[13, 14]",37,"[1397, 14814, 2004, 2006, 2031, 24746, 3, 3134..."
396,2,"[0, 5]",35,"[12, 1397, 14814, 15023, 15024, 15285, 15704, ..."
443,2,"[61, 13]",33,"[1397, 14814, 15944, 18411, 2004, 2006, 2031, ..."
354,2,"[13, 13]",33,"[1397, 14814, 2004, 2006, 2031, 22362, 22363, ..."
335,2,"[5, 0]",32,"[12, 1397, 14814, 15023, 15024, 15285, 15704, ..."
81,12,"[13, 13, 14, 9, 14, 56, 11, 14, 61, 13, 13, 14]",29,"[1397, 14814, 2004, 2006, 2031, 24746, 3, 3134..."
243,3,"[12, 12, 257]",29,"[1397, 14814, 15023, 15024, 15285, 15704, 1577..."


In [18]:
# num_of_exploit 기준으로 sorting해서 출력할 df 저장
soft_most_frequency_ngramdf = soft_dedup_ngramdf.sort_values(by=["num_of_exploit"], ascending=False).head(50)
soft_most_frequency_ngramdf = soft_most_frequency_ngramdf[['N', 'Ngram_syscall','num_of_exploit']]

# system call 번호 -> 이름으로 변경
for index, row in soft_most_frequency_ngramdf.iterrows():
    Ngram_sysname = list()
    for syscall in row.Ngram_syscall:
        Ngram_sysname.append(convert_num_name(syscall))
        
    soft_most_frequency_ngramdf['Ngram_syscall'][index] = Ngram_sysname

In [19]:
# Excel로 저장
soft_most_frequency_ngramdf.to_excel("top50ngram_in_all.xlsx")
soft_most_frequency_ngramdf.head(10)

,N,Ngram_syscall,num_of_exploit
459,2,"[brk, brk]",55
455,2,"[openat, fstat]",40
399,2,"[read, close]",38
338,2,"[rt_sigaction, rt_sigprocmask]",37
396,2,"[read, fstat]",35
443,2,"[wait4, rt_sigaction]",33
354,2,"[rt_sigaction, rt_sigaction]",33
335,2,"[fstat, read]",32
81,12,"[rt_sigaction, rt_sigaction, rt_sigprocmask, m...",29
243,3,"[brk, brk, openat]",29


# 가장 길이가 긴 N-gram
## 상위 10개

In [20]:
soft_longest_ngramdf = soft_dedup_ngramdf.sort_values(by=["N"], ascending=False).head(10)
# system call 번호 -> 이름으로 변경
for index, row in soft_longest_ngramdf.iterrows():
    Ngram_sysname = list()
    for syscall in row.Ngram_syscall:
        Ngram_sysname.append(convert_num_name(syscall))
    soft_longest_ngramdf['Ngram_syscall'][index] = Ngram_sysname
soft_longest_ngramdf.to_excel("top10_longest_ngram_in_all(soft).xlsx")

In [21]:
soft_longest_ngramdf.head(15)

,N,Ngram_syscall,num_of_exploit,EIDs
0,61,"[socket, add_key, add_key, add_key, setsockopt...",2,"[40871, 47170]"
1,49,"[clock_nanosleep, read, close, fstat, mmap, mp...",2,"[40871, 47170]"
2,45,"[brk, brk, fstat, read, openat, stat, brk, brk...",2,"[47163, 50541]"
3,43,"[mmap, read, close, fstat, mmap, mprotect, brk...",2,"[40616, 43127]"
4,43,"[brk, brk, fstat, read, openat, stat, brk, brk...",2,"[47163, 50541]"
5,42,"[read, close, fstat, mmap, mprotect, brk, rt_s...",3,"[40616, 43127, 44302]"
6,39,"[brk, brk, fstat, read, openat, stat, fstat, r...",2,"[47163, 50541]"
7,38,"[brk, brk, openat, brk, brk, openat, fstat, re...",2,"[14814, 15023]"
8,35,"[brk, brk, openat, brk, brk, openat, fstat, re...",2,"[14814, 15023]"
9,34,"[brk, brk, openat, fstat, read, fstat, read, f...",2,[15023]


## 상위 200개

In [22]:
soft_longest_ngramdf = soft_dedup_ngramdf.sort_values(by=["N"], ascending=False).head(100)
# system call 번호 -> 이름으로 변경
for index, row in soft_longest_ngramdf.iterrows():
    Ngram_sysname = list()
    for syscall in row.Ngram_syscall:
        Ngram_sysname.append(convert_num_name(syscall))
    soft_longest_ngramdf['Ngram_syscall'][index] = Ngram_sysname
soft_longest_ngramdf.to_excel("top200_longest_ngram_in_all.xlsx")

In [23]:
# Syscall Number 버전
soft_longest_ngramdf = soft_dedup_ngramdf.sort_values(by=["N"], ascending=False).head(200)
soft_longest_ngramdf.to_excel("top200_longest_ngram_in_all_sysnum_version.xlsx")

# Token Size별 N-gram과 exploit 수

In [24]:
nsize_nnum_enum_soft = pd.DataFrame(columns=["N", "NgramNum","exploitNum"])
ngram_data = {"N":"","NgramNum":0,"exploitNum":0, "exploits":set()}

for N in list(set(soft_dedup_ngramdf["N"])):
    ngram_data["N"] = N
    for index, row in soft_dedup_ngramdf.query(f'N == {N}').iterrows():
        ngram_data["NgramNum"] += 1
        for EID in row.EIDs:
            ngram_data["exploits"].add(EID)
    ngram_data["exploitNum"] = len(ngram_data["exploits"])
    nsize_nnum_enum_soft = nsize_nnum_enum_soft.append(ngram_data, ignore_index=True)
    ngram_data = {"N":"","NgramNum":0,"exploitNum":0, "exploits":set()}
    
nsize_nnum_enum_soft.head()

,N,NgramNum,exploitNum,exploits
0,2,163,105,"{44302, 37167, 22362, 44299, 45516, 8673, 2455..."
1,3,108,92,"{44302, 37167, 22362, 44299, 8673, 45516, 6851..."
2,4,52,78,"{44302, 37167, 22362, 45516, 8673, 6851, 42276..."
3,5,18,49,"{44302, 15024, 43029, 33516, 9083, 15704, 895,..."
4,6,20,54,"{15024, 43029, 33516, 9083, 15704, 895, 47169,..."


In [25]:
# EXCEL로 저장
nsize_nnum_enum_soft[['N','NgramNum','exploitNum']].to_excel("ngram_num_and_exploit_num_per_ngram_token_size.xlsx")

# 각 N-gram 별 최다 Exploit 출현 N-gram Syscall Sequence Pattern

In [26]:
import pandas as pd
soft_max_exploit_ngram = pd.DataFrame()
# 최다 출현 N-gram 모으기
for N in list(set(soft_dedup_ngramdf["N"])):
    soft_max_exploit_ngram = soft_max_exploit_ngram.append(soft_dedup_ngramdf.query(f'N == {N}').sort_values(by=["num_of_exploit"], ascending=False, ignore_index=True).iloc[0])
    
soft_max_exploit_ngram = soft_max_exploit_ngram.reset_index(drop=True)

# Sysnum 버전 저장
soft_max_exploit_ngram.to_excel("max_exploit_ngram(soft_sysnum).xlsx")

# Syscal Sequence의 syscall number -> syscall name으로 변환
for index, row in soft_max_exploit_ngram.iterrows():
    Ngram_sysname = list()
    for syscall in row.Ngram_syscall:
        Ngram_sysname.append(convert_num_name(syscall))
    soft_max_exploit_ngram['Ngram_syscall'][index] = Ngram_sysname
soft_max_exploit_ngram.head()

/tmp/ipykernel_1611693/3516025826.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  soft_max_exploit_ngram['Ngram_syscall'][index] = Ngram_sysname


,N,Ngram_syscall,num_of_exploit,EIDs
0,2.0,"[brk, brk]",55.0,"[10613, 12, 1397, 14814, 15023, 15024, 15285, ..."
1,3.0,"[brk, brk, openat]",29.0,"[1397, 14814, 15023, 15024, 15285, 15704, 1577..."
2,4.0,"[brk, brk, openat, fstat]",26.0,"[1397, 14814, 15023, 15024, 15285, 15704, 1577..."
3,5.0,"[brk, brk, openat, fstat, read]",24.0,"[1397, 14814, 15023, 15024, 15285, 15704, 1577..."
4,6.0,"[fstat, read, fstat, read, fstat, read]",27.0,"[12, 14814, 15023, 15024, 15285, 15704, 15774,..."


In [27]:
# Excel에 저장
soft_max_exploit_ngram.to_excel("max_exploit_ngram.xlsx")

# 큰 Ngram부터 포함 시 전체 exploit의 coverage
## Soft

In [28]:
soft_dedup_ngramdf.head()

,N,Ngram_syscall,num_of_exploit,EIDs
0,61,"[41, 248, 248, 248, 54, 0, 3, 5, 9, 10, 12, 13...",2,"[40871, 47170]"
1,49,"[230, 0, 3, 5, 9, 10, 12, 13, 14, 17, 56, 218,...",2,"[40871, 47170]"
2,45,"[12, 12, 5, 0, 257, 4, 12, 12, 5, 0, 257, 4, 1...",2,"[47163, 50541]"
3,43,"[9, 0, 3, 5, 9, 10, 12, 13, 14, 17, 56, 218, 2...",2,"[40616, 43127]"
4,43,"[12, 12, 5, 0, 257, 4, 12, 12, 5, 0, 257, 4, 1...",2,"[47163, 50541]"


In [29]:
len(soft_dedup_ngramdf)

478

In [30]:
TOTAL_EXPLOIT_NUM = 106

coveragedf = pd.DataFrame(columns=["N","NgramNum","coverage"])
coverage_eid = set()
Ngram_Num = 0
tmpdict = {"N":"","NgramNum":0,"coverage":0.0}
for index, row in soft_dedup_ngramdf.iterrows():
    if row.N in coveragedf.N.to_list():
        Ngram_Num += 1
        coveragedf[coveragedf["N"] == row.N].NgramNum = Ngram_Num  
        for eid in row.EIDs:
            coverage_eid.add(eid)
        coveragedf[coveragedf["N"] == row.N].coverage = round((len(coverage_eid) * 100)/TOTAL_EXPLOIT_NUM)
        

In [31]:
TOTAL_EXPLOIT_NUM = 106

coveragedf = pd.DataFrame(columns=["N","NgramNum","coverage"])
coverage_eid = set()
Ngram_Num = 0
total_Num= 0
tmpdict = {"N":"","NgramNum":0,"coverage":0.0}
for index, row in soft_dedup_ngramdf.iterrows():
    total_Num += 1
    if row.N in coveragedf.N.to_list():
        Ngram_Num += 1
        coveragedf["NgramNum"][coveragedf["N"] == row.N]= Ngram_Num
        for eid in row.EIDs:
            coverage_eid.add(eid)
        coveragedf["coverage"][coveragedf["N"] == row.N]= round((len(coverage_eid) * 100)/TOTAL_EXPLOIT_NUM)
    else:
        Ngram_Num += 1
        tmpdict["N"] = row.N
        for eid in row.EIDs:
            coverage_eid.add(eid)
        tmpdict["coverage"] = round((len(coverage_eid) * 100)/TOTAL_EXPLOIT_NUM)
        tmpdict["NgramNum"] = Ngram_Num
        coveragedf = coveragedf.append(tmpdict, ignore_index=True)
        tmpdict = {"N":"","NgramNum":0,"coverage":""}

coveragedf.to_excel("ngram_exploit_coverage.xlsx")

### Pattern의 총 개수 (길이 10 이상)

In [32]:
for index, row in soft_dedup_ngramdf.iterrows():
    if not len(row.Ngram_syscall) == row.N:
        soft_dedup_ngramdf["N"][index] = len(row.Ngram_syscall)
soft_dedup_ngramdf.sort_values(by=["N"], ascending=False)
soft_dedup_ngramdf.head()

,N,Ngram_syscall,num_of_exploit,EIDs
0,61,"[41, 248, 248, 248, 54, 0, 3, 5, 9, 10, 12, 13...",2,"[40871, 47170]"
1,49,"[230, 0, 3, 5, 9, 10, 12, 13, 14, 17, 56, 218,...",2,"[40871, 47170]"
2,45,"[12, 12, 5, 0, 257, 4, 12, 12, 5, 0, 257, 4, 1...",2,"[47163, 50541]"
3,43,"[9, 0, 3, 5, 9, 10, 12, 13, 14, 17, 56, 218, 2...",2,"[40616, 43127]"
4,43,"[12, 12, 5, 0, 257, 4, 12, 12, 5, 0, 257, 4, 1...",2,"[47163, 50541]"


In [33]:
more_10_cnt = 0
for index, row in soft_dedup_ngramdf.iterrows():
    if row.N >= 10:
        more_10_cnt += 1
print("total pattern #: ",len(soft_dedup_ngramdf))
print("10+ pattern #: ",more_10_cnt)

total pattern #:  478
10+ pattern #:  94


# 카테고리별 분석
## Kernel Component 별 N-gram 분석

In [35]:
# 익스플로잇 Category 정보 load
import pandas as pd
allExploitInfo = pd.read_pickle("allExploitInfo.pkl")
allExploitInfo.head()

,EID,CVEID,CVE_des,file,function
0,3,2003-0127,The kernel module loader in Linux kernel 2.2.x...,None,None
1,12,2003-0127,The kernel module loader in Linux kernel 2.2.x...,None,None
2,131,2003-0961,Integer overflow in the do_brk function for th...,None,do_brk
3,145,2003-0985,The mremap system call (do_mremap) in Linux ke...,None,None
4,160,2004-0077,The do_mremap function for the mremap system c...,None,do_munmap


In [36]:
# 익스플로잇 카테고리 정보 load
import pandas as pd
allExploitCatInfo = pd.read_pickle("onlyCategoryWithEID.pkl")
allExploitCatInfo.head()

,EID,Kernel_Component,Vulnerability_Type
0,2013,fs,BA
1,4756,net,BA
2,8678,kernel,RC
3,10613,fs,ETC
4,15023,arch,BA


### Kernel Component의 N-gram 분석 Return

In [37]:
def kernel_component_ngram(kernel_component_EID, kernel_component_name):
    kernel_component_ngram = dict()
    
    for index, row in soft_dedup_ngramdf.iterrows():
        for EID in kernel_component_EID:
            if str(EID) in row.EIDs:
                if ",".join(row.Ngram_syscall) in kernel_component_ngram.keys():
                    kernel_component_ngram[",".join(row.Ngram_syscall)] += 1
                else:
                    kernel_component_ngram[",".join(row.Ngram_syscall)] = 1
    
    # sysnum -> sysname
    kernel_component_ngram_sysname = dict()
    for ngram, cnt in kernel_component_ngram.items():
        sys_name_list = list()
        for syscall in ngram.split(","):
            sys_name_list.append(convert_num_name(syscall))
        kernel_component_ngram_sysname[",".join(sys_name_list)] = kernel_component_ngram[ngram]
    
    # 중복 제거
    sorted_kernel_component_ngram_sysname = sorted(kernel_component_ngram_sysname.items(), key=lambda x: x[1], reverse=True)
    sorted_kernel_component_ngram_sysname = list(map(lambda x: list(x), sorted_kernel_component_ngram_sysname))
    for i, ngram1 in enumerate(sorted_kernel_component_ngram_sysname):
        if ngram1 == "": continue
        for j, ngram2 in enumerate(sorted_kernel_component_ngram_sysname[i+1:]):
            if ngram2 == "": continue
            if ngram2[0] in ngram1[0]:
                sorted_kernel_component_ngram_sysname[i+j+1][0] = ""
    
    final_kernel_component_ngram = list()
    for ngram in sorted_kernel_component_ngram_sysname:
        if not ngram[0] == "":
            final_kernel_component_ngram.append(ngram)
            
    final_kernel_component_ngram_df = pd.DataFrame(columns=["N","ngram","percent"])
    row = {"ngram":"", "percent":0, "N":0, "cnt":0}
    for ngram in final_kernel_component_ngram:
        row["ngram"] = ngram[0]
        row["N"] = len(ngram[0].split(","))
        row["cnt"] =  ngram[1]
        row["percent"] = ngram[1]/len(kernel_component_EID)*100
        final_kernel_component_ngram_df = final_kernel_component_ngram_df.append(row, ignore_index=True)
        row = {"ngram":"", "percent":0, "N":0, "cnt":0}
    
    final_kernel_component_ngram_df.to_excel(f"kernel_component_{kernel_component_name}_ngram.xlsx")
    print(f"Finished making {kernel_component_name} ngram excel! - total EID #: {len(kernel_component_EID)}")
       

### Kernel Component 카테고리별 N-gram 분석 결과 저장

In [38]:
allExploitCatInfo.head()

,EID,Kernel_Component,Vulnerability_Type
0,2013,fs,BA
1,4756,net,BA
2,8678,kernel,RC
3,10613,fs,ETC
4,15023,arch,BA


In [39]:
# 취약점 카테고리 기준으로 Exploit Code 모으기
comCat = list(set(allExploitCatInfo["Kernel_Component"].to_list()))
comtoexploit = pd.DataFrame()
comtoexploit["Kernel_Component"] = comCat
comtoexploit["EIDs"] = ""

for index, row in comtoexploit.iterrows():
    comtoexploit["EIDs"][index] = []

for index1, row1 in comtoexploit.iterrows():
    for index2, row2 in allExploitCatInfo.iterrows():
        if row1.Kernel_Component == row2.Kernel_Component:
            # 첫 번째 추가일 경우
            if comtoexploit["EIDs"][index1] == np.nan:
                comtoexploit["EIDs"][index1] = [row2.EID]
            # 두 번째부터 추가일 경우
            else:
                comtoexploit["EIDs"][index1].append(row2.EID)
                
# EID 중복 제거
for index, row in comtoexploit.iterrows():
    comtoexploit["EIDs"][index] = list(set(row.EIDs))

comtoexploit.head()

,Kernel_Component,EIDs
0,drivers,"[33516, 37167, 9083, 15774, 37183]"
1,kernel,"[3, 44298, 12, 26131, 43029, 20000, 33824, 353..."
2,ipc,"[45553, 17787]"
3,arch,"[27297, 30604, 4460, 15023, 31574, 37722]"
4,security,"[40003, 39277]"


In [40]:
print(comtoexploit[comtoexploit.Kernel_Component=="lib"].EIDs.values[0])

[42275, 42276]


In [41]:
for index, row in comtoexploit.iterrows():
    kernel_component_ngram(row.EIDs, row.Kernel_Component)

Finished making drivers ngram excel! - total EID #: 5
Finished making kernel ngram excel! - total EID #: 24
Finished making ipc ngram excel! - total EID #: 2
Finished making arch ngram excel! - total EID #: 6
Finished making security ngram excel! - total EID #: 2
Finished making include ngram excel! - total EID #: 3
Finished making mm ngram excel! - total EID #: 5
Finished making fs ngram excel! - total EID #: 29
Finished making net ngram excel! - total EID #: 32
Finished making lib ngram excel! - total EID #: 2


### 전체익스플로잇 비율 추가

In [42]:
def add_total_percent(kc_name):
    kernel_component_ngramdf = pd.read_excel(f"kernel_component_{kc_name}_ngram.xlsx")
    kernel_component_ngramdf = kernel_component_ngramdf.loc[:, ~kernel_component_ngramdf.columns.str.contains('^Unnamed')]

    for index, row in kernel_component_ngramdf.iterrows():
        ngram_list=  row.ngram.split(",")
        new_ngram_list = list()
        for syscall in ngram_list:
            new_ngram_list.append(convert_name_num(syscall))
        kernel_component_ngramdf["ngram"][index] = ",".join(new_ngram_list)

    tmpdf = soft_dedup_ngramdf.copy()
    for index, row in tmpdf.iterrows():
        tmpdf["Ngram_syscall"][index] = ",".join(row.Ngram_syscall)

    kernel_component_ngramdf["total_percent"] = [0 for i in range(len(kernel_component_ngramdf))]
    kernel_component_ngramdf["total_cnt"] = [0 for i in range(len(kernel_component_ngramdf))]

    for index, row in kernel_component_ngramdf.iterrows():
        total_ngram_eids = tmpdf["EIDs"][tmpdf["Ngram_syscall"]==row.ngram].to_list()
        total_ngram_eids = set(sum(total_ngram_eids, []))
        kernel_component_ngramdf["total_cnt"][index] = len(total_ngram_eids)
        kernel_component_ngramdf["total_percent"][index] = (row.cnt/len(total_ngram_eids))*100
    
    for index, row in kernel_component_ngramdf.iterrows():
        sysname_syspath = list()
        ngram = row.ngram.split(",")
        for syscall in ngram:
            sysname_syspath.append(convert_num_name(syscall))
        kernel_component_ngramdf["ngram"][index] = sysname_syspath
    
    kernel_component_ngramdf.to_excel(f"kernel_component_{kc_name}_ngram_with_total.xlsx")
    print(f"Finished making {kc_name} ngram excel!")   

In [52]:
for index, row in comtoexploit.iterrows():
    add_total_percent(row.Kernel_Component)

/tmp/ipykernel_1611693/2855002658.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kernel_component_ngramdf["ngram"][index] = ",".join(new_ngram_list)
/tmp/ipykernel_1611693/2855002658.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kernel_component_ngramdf["total_cnt"][index] = len(total_ngram_eids)
/tmp/ipykernel_1611693/2855002658.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kernel_component_ngramdf["total_percent"]

Finished making drivers ngram excel!
Finished making kernel ngram excel!
Finished making ipc ngram excel!
Finished making arch ngram excel!
Finished making security ngram excel!
Finished making include ngram excel!
Finished making mm ngram excel!
Finished making fs ngram excel!
Finished making net ngram excel!
Finished making lib ngram excel!


## 취약점 유형별 N-gram 분석

In [53]:
allExploitCatInfo.head()

,EID,Kernel_Component,Vulnerability_Type
0,2013,fs,BA
1,4756,net,BA
2,8678,kernel,RC
3,10613,fs,ETC
4,15023,arch,BA


In [54]:
# 취약점 카테고리 기준으로 Exploit Code 모으기
vulCat = list(set(allExploitCatInfo["Vulnerability_Type"].to_list()))
vultoexploit = pd.DataFrame()
vultoexploit["vul_cat"] = vulCat
vultoexploit["EIDs"] = ""

for index, row in vultoexploit.iterrows():
    vultoexploit["EIDs"][index] = []

for index1, row1 in vultoexploit.iterrows():
    for index2, row2 in allExploitCatInfo.iterrows():
        if row1.vul_cat == row2.Vulnerability_Type:
            # 첫 번째 추가일 경우
            if vultoexploit["EIDs"][index1] == np.nan:
                vultoexploit["EIDs"][index1] = [row2.EID]
            # 두 번째부터 추가일 경우
            else:
                vultoexploit["EIDs"][index1].append(row2.EID)
                
# EID 중복 제거
for index, row in vultoexploit.iterrows():
    vultoexploit["EIDs"][index] = list(set(row.EIDs))

vultoexploit.head()

,vul_cat,EIDs
0,RC,"[40839, 44302, 43418, 40871, 40616, 33321, 333..."
1,IV,"[5092, 5093, 25288, 25289, 24746, 24555, 44299..."
2,PU,"[47170, 9542, 9575, 40871, 33321, 33322, 50808..."
3,BA,"[42887, 9479, 40839, 41994, 41995, 44300, 3060..."
4,ETC,[10613]


### 취약점 카테고리별 N-gram 분석 결과 저장 함수

In [55]:
def vulnerability_ngram(vulnerability_EID, vulnerability_name):
    vulnerability_ngram = dict()
    
    for index, row in soft_dedup_ngramdf.iterrows():
        for EID in vulnerability_EID:
            if str(EID) in row.EIDs:
                if ",".join(row.Ngram_syscall) in vulnerability_ngram.keys():
                    vulnerability_ngram[",".join(row.Ngram_syscall)] += 1
                else:
                    vulnerability_ngram[",".join(row.Ngram_syscall)] = 1

    # sysnum -> sysname
    vulnerability_ngram_sysname = dict()
    for ngram, cnt in vulnerability_ngram.items():
        sys_name_list = list()
        for syscall in ngram.split(","):
            sys_name_list.append(convert_num_name(syscall))
        vulnerability_ngram_sysname[",".join(sys_name_list)] = vulnerability_ngram[ngram]
    
    # 중복 제거
    sorted_vulnerability_ngram_sysname = sorted(vulnerability_ngram_sysname.items(), key=lambda x: x[1], reverse=True)
    sorted_vulnerability_ngram_sysname = list(map(lambda x: list(x), sorted_vulnerability_ngram_sysname))
    for i, ngram1 in enumerate(sorted_vulnerability_ngram_sysname):
        if ngram1 == "": continue
        for j, ngram2 in enumerate(sorted_vulnerability_ngram_sysname[i+1:]):
            if ngram2 == "": continue
            if ngram2[0] in ngram1[0]:
                sorted_vulnerability_ngram_sysname[i+j+1][0] = ""
    
    final_vulnerability_ngram = list()
    for ngram in sorted_vulnerability_ngram_sysname:
        if not ngram[0] == "":
            final_vulnerability_ngram.append(ngram)
            
    final_vulnerability_ngram_df = pd.DataFrame(columns=["N","ngram","percent","cnt"])
    row = {"ngram":"", "percent":0, "N":0,"cnt":0}
    for ngram in final_vulnerability_ngram:
        row["ngram"] = ngram[0]
        row["N"] = len(ngram[0].split(","))
        row["cnt"] = ngram[1]
        row["percent"] = ngram[1]/len(vulnerability_EID)*100
        final_vulnerability_ngram_df = final_vulnerability_ngram_df.append(row, ignore_index=True)
        row = {"ngram":"", "percent":0, "N":0,"cnt":0}
    
    final_vulnerability_ngram_df.to_excel(f"vulnerability_{vulnerability_name}_ngram.xlsx")
    print(f"Finished making {vulnerability_name} ngram excel! - total EID #: {len(vulnerability_EID)}")
       

### 취약점 카테고리별 N-gram 분석 결과 저장

In [56]:
for index, row in vultoexploit.iterrows():
    if row.vul_cat == "ETC":
        continue
    vulnerability_ngram(list(set(row.EIDs)), row.vul_cat)

Finished making RC ngram excel! - total EID #: 19
Finished making IV ngram excel! - total EID #: 17
Finished making PU ngram excel! - total EID #: 14
Finished making BA ngram excel! - total EID #: 26
Finished making AC ngram excel! - total EID #: 33
Finished making IL ngram excel! - total EID #: 5
Finished making IC ngram excel! - total EID #: 17


In [57]:
def add_total_percent_for_vul(kc_name):
    kernel_component_ngramdf = pd.read_excel(f"vulnerability_{kc_name}_ngram.xlsx")
    kernel_component_ngramdf = kernel_component_ngramdf.loc[:, ~kernel_component_ngramdf.columns.str.contains('^Unnamed')]

    for index, row in kernel_component_ngramdf.iterrows():
        ngram_list=  row.ngram.split(",")
        new_ngram_list = list()
        for syscall in ngram_list:
            new_ngram_list.append(convert_name_num(syscall))
        kernel_component_ngramdf["ngram"][index] = ",".join(new_ngram_list)

    tmpdf = soft_dedup_ngramdf.copy()
    for index, row in tmpdf.iterrows():
        tmpdf["Ngram_syscall"][index] = ",".join(row.Ngram_syscall)

    kernel_component_ngramdf["total_percent"] = [0 for i in range(len(kernel_component_ngramdf))]
    kernel_component_ngramdf["total_cnt"] = [0 for i in range(len(kernel_component_ngramdf))]

    for index, row in kernel_component_ngramdf.iterrows():
        total_ngram_eids = tmpdf["EIDs"][tmpdf["Ngram_syscall"]==row.ngram].to_list()
        total_ngram_eids = set(sum(total_ngram_eids, []))
        kernel_component_ngramdf["total_cnt"][index] = len(total_ngram_eids)
        kernel_component_ngramdf["total_percent"][index] = (row.cnt/len(total_ngram_eids))*100
    
    for index, row in kernel_component_ngramdf.iterrows():
        sysname_syspath = list()
        ngram = row.ngram.split(",")
        for syscall in ngram:
            sysname_syspath.append(convert_num_name(syscall))
        kernel_component_ngramdf["ngram"][index] = sysname_syspath
    
    kernel_component_ngramdf.to_excel(f"vulnerability_{kc_name}_ngram_with_total.xlsx")
    print(f"Finished making {kc_name} ngram excel!")   

In [58]:
for index, row in vultoexploit.iterrows():
    if row.vul_cat == "ETC":
        continue
    add_total_percent_for_vul(row.vul_cat)

/tmp/ipykernel_1611693/1633394789.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kernel_component_ngramdf["ngram"][index] = ",".join(new_ngram_list)
/tmp/ipykernel_1611693/1633394789.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kernel_component_ngramdf["total_cnt"][index] = len(total_ngram_eids)
/tmp/ipykernel_1611693/1633394789.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kernel_component_ngramdf["total_percent"]

Finished making RC ngram excel!
Finished making IV ngram excel!
Finished making PU ngram excel!
Finished making BA ngram excel!
Finished making AC ngram excel!
Finished making IL ngram excel!
Finished making IC ngram excel!


## 취약점 & 커널 컴포넌트별 N-gram 분석
- 익스플로잇이 10개 이상인 카테고리: 4개
- 익스플로잇이 5개 이상인 카테고리: +5개

In [59]:
def combine_ngram(combine_EID, combine_name):
    vulnerability_ngram = dict()
    
    for index, row in soft_dedup_ngramdf.iterrows():
        for EID in combine_EID:
            if str(EID) in row.EIDs:
                if ",".join(row.Ngram_syscall) in vulnerability_ngram.keys():
                    vulnerability_ngram[",".join(row.Ngram_syscall)] += 1
                else:
                    vulnerability_ngram[",".join(row.Ngram_syscall)] = 1

    # sysnum -> sysname
    vulnerability_ngram_sysname = dict()
    for ngram, cnt in vulnerability_ngram.items():
        sys_name_list = list()
        for syscall in ngram.split(","):
            sys_name_list.append(convert_num_name(syscall))
        vulnerability_ngram_sysname[",".join(sys_name_list)] = vulnerability_ngram[ngram]
    
    # 중복 제거
    sorted_vulnerability_ngram_sysname = sorted(vulnerability_ngram_sysname.items(), key=lambda x: x[1], reverse=True)
    sorted_vulnerability_ngram_sysname = list(map(lambda x: list(x), sorted_vulnerability_ngram_sysname))
    for i, ngram1 in enumerate(sorted_vulnerability_ngram_sysname):
        if ngram1 == "": continue
        for j, ngram2 in enumerate(sorted_vulnerability_ngram_sysname[i+1:]):
            if ngram2 == "": continue
            if ngram2[0] in ngram1[0]:
                sorted_vulnerability_ngram_sysname[i+j+1][0] = ""
    
    final_vulnerability_ngram = list()
    for ngram in sorted_vulnerability_ngram_sysname:
        if not ngram[0] == "":
            final_vulnerability_ngram.append(ngram)
            
    final_vulnerability_ngram_df = pd.DataFrame(columns=["N","ngram","percent","cnt"])
    row = {"ngram":"", "percent":0, "N":0,"cnt":0}
    for ngram in final_vulnerability_ngram:
        row["ngram"] = ngram[0]
        row["N"] = len(ngram[0].split(","))
        row["cnt"] = ngram[1]
        row["percent"] = ngram[1]/len(combine_EID)*100
        final_vulnerability_ngram_df = final_vulnerability_ngram_df.append(row, ignore_index=True)
        row = {"ngram":"", "percent":0, "N":0,"cnt":0}
    
    final_vulnerability_ngram_df.to_excel(f"combination_{combine_name}_ngram.xlsx")
    print(f"Finished making {combine_name} ngram excel! - total EID #: {len(combine_EID)}")
       

In [60]:
def add_total_percent_for_combine(kc_name):
    kernel_component_ngramdf = pd.read_excel(f"combination_{kc_name}_ngram.xlsx")
    kernel_component_ngramdf = kernel_component_ngramdf.loc[:, ~kernel_component_ngramdf.columns.str.contains('^Unnamed')]

    for index, row in kernel_component_ngramdf.iterrows():
        ngram_list=  row.ngram.split(",")
        new_ngram_list = list()
        for syscall in ngram_list:
            new_ngram_list.append(convert_name_num(syscall))
        kernel_component_ngramdf["ngram"][index] = ",".join(new_ngram_list)

    tmpdf = soft_dedup_ngramdf.copy()
    for index, row in tmpdf.iterrows():
        tmpdf["Ngram_syscall"][index] = ",".join(row.Ngram_syscall)

    kernel_component_ngramdf["total_percent"] = [0 for i in range(len(kernel_component_ngramdf))]
    kernel_component_ngramdf["total_cnt"] = [0 for i in range(len(kernel_component_ngramdf))]

    for index, row in kernel_component_ngramdf.iterrows():
        total_ngram_eids = tmpdf["EIDs"][tmpdf["Ngram_syscall"]==row.ngram].to_list()
        total_ngram_eids = set(sum(total_ngram_eids, []))
        kernel_component_ngramdf["total_cnt"][index] = len(total_ngram_eids)
        kernel_component_ngramdf["total_percent"][index] = (row.cnt/len(total_ngram_eids))*100
    
    for index, row in kernel_component_ngramdf.iterrows():
        sysname_syspath = list()
        ngram = row.ngram.split(",")
        for syscall in ngram:
            sysname_syspath.append(convert_num_name(syscall))
        kernel_component_ngramdf["ngram"][index] = sysname_syspath
    
    kernel_component_ngramdf.to_excel(f"combination_{kc_name}_ngram_with_total.xlsx")
    print(f"Finished making {kc_name} ngram excel!")   

In [61]:
allExploitCatInfo.head()

,EID,Kernel_Component,Vulnerability_Type
0,2013,fs,BA
1,4756,net,BA
2,8678,kernel,RC
3,10613,fs,ETC
4,15023,arch,BA


#### AC & fs

In [62]:
# AC 취약점 카테고리와 fs 컴포넌트에 속하는 익스플로잇 모으기
mask = list()
for index, row in allExploitCatInfo.iterrows():
    if row.Kernel_Component == "fs" and row.Vulnerability_Type == "AC":
        mask.append(True)
    else:
        mask.append(False)
        
maskedExploitCatInfo = allExploitCatInfo[mask]
maskedExploitCatInfo

,EID,Kernel_Component,Vulnerability_Type
28,18411,fs,AC
31,2004,fs,AC
32,2005,fs,AC
33,2006,fs,AC
34,2031,fs,AC
61,34923,fs,AC
62,35161,fs,AC
64,37292,fs,AC
66,39166,fs,AC
67,39230,fs,AC


In [63]:
combine_ngram(list(set(maskedExploitCatInfo.EID.tolist())), "AC_fs")
add_total_percent_for_combine("AC_fs")

Finished making AC_fs ngram excel! - total EID #: 12


/tmp/ipykernel_1611693/684178284.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kernel_component_ngramdf["ngram"][index] = ",".join(new_ngram_list)
/tmp/ipykernel_1611693/684178284.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kernel_component_ngramdf["total_cnt"][index] = len(total_ngram_eids)
/tmp/ipykernel_1611693/684178284.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kernel_component_ngramdf["total_percent"][in

Finished making AC_fs ngram excel!


#### AC & kernel

In [64]:
# AC 취약점 카테고리와 kernel 컴포넌트에 속하는 익스플로잇 모으기
mask = list()
for index, row in allExploitCatInfo.iterrows():
    if row.Kernel_Component == "kernel" and row.Vulnerability_Type == "AC":
        mask.append(True)
    else:
        mask.append(False)
        
maskedExploitCatInfo = allExploitCatInfo[mask]
maskedExploitCatInfo

,EID,Kernel_Component,Vulnerability_Type
11,38390,kernel,AC
15,12,kernel,AC
30,20000,kernel,AC
37,22362,kernel,AC
38,22363,kernel,AC
45,25450,kernel,AC
48,3,kernel,AC
59,33824,kernel,AC
63,35370,kernel,AC
94,47163,kernel,AC


In [65]:
combine_ngram(list(set(maskedExploitCatInfo.EID.tolist())), "AC_kernel")
add_total_percent_for_combine("AC_kernel")

Finished making AC_kernel ngram excel! - total EID #: 12


/tmp/ipykernel_1611693/684178284.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kernel_component_ngramdf["ngram"][index] = ",".join(new_ngram_list)
/tmp/ipykernel_1611693/684178284.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kernel_component_ngramdf["total_cnt"][index] = len(total_ngram_eids)
/tmp/ipykernel_1611693/684178284.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kernel_component_ngramdf["total_percent"][in

Finished making AC_kernel ngram excel!


#### IV & net

In [66]:
# IV 취약점 카테고리와 net 컴포넌트에 속하는 익스플로잇 모으기
mask = list()
for index, row in allExploitCatInfo.iterrows():
    if row.Kernel_Component == "net" and row.Vulnerability_Type == "IV":
        mask.append(True)
    else:
        mask.append(False)
        
maskedExploitCatInfo = allExploitCatInfo[mask]
maskedExploitCatInfo

,EID,Kernel_Component,Vulnerability_Type
19,15285,net,IV
39,24555,net,IV
40,24746,net,IV
42,25288,net,IV
43,25289,net,IV
50,31347,net,IV
55,33336,net,IV
87,44299,net,IV
109,8572,net,IV
113,926,net,IV


In [67]:
combine_ngram(list(set(maskedExploitCatInfo.EID.tolist())), "IV_net")
add_total_percent_for_combine("IV_net")

Finished making IV_net ngram excel! - total EID #: 10


/tmp/ipykernel_1611693/684178284.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kernel_component_ngramdf["ngram"][index] = ",".join(new_ngram_list)
/tmp/ipykernel_1611693/684178284.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kernel_component_ngramdf["total_cnt"][index] = len(total_ngram_eids)
/tmp/ipykernel_1611693/684178284.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kernel_component_ngramdf["total_percent"][in

Finished making IV_net ngram excel!


### BA & net

In [68]:
# BA 취약점 카테고리와 net 컴포넌트에 속하는 익스플로잇 모으기
mask = list()
for index, row in allExploitCatInfo.iterrows():
    if row.Kernel_Component == "net" and row.Vulnerability_Type == "BA":
        mask.append(True)
    else:
        mask.append(False)
        
maskedExploitCatInfo = allExploitCatInfo[mask]
maskedExploitCatInfo

,EID,Kernel_Component,Vulnerability_Type
1,4756,net,BA
12,40049,net,BA
13,44300,net,BA
25,17787,net,BA
76,41995,net,BA
98,50135,net,BA
115,9479,net,BA
117,9545,net,BA
120,15916,net,BA
122,15944,net,BA


In [69]:
combine_ngram(list(set(maskedExploitCatInfo.EID.tolist())), "BA_net")
add_total_percent_for_combine("BA_net")

Finished making BA_net ngram excel! - total EID #: 14


/tmp/ipykernel_1611693/684178284.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kernel_component_ngramdf["ngram"][index] = ",".join(new_ngram_list)
/tmp/ipykernel_1611693/684178284.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kernel_component_ngramdf["total_cnt"][index] = len(total_ngram_eids)
/tmp/ipykernel_1611693/684178284.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kernel_component_ngramdf["total_percent"][in

Finished making BA_net ngram excel!


### RC & fs

In [70]:
# RC 취약점 카테고리와 fs 컴포넌트에 속하는 익스플로잇 모으기
mask = list()
for index, row in allExploitCatInfo.iterrows():
    if row.Kernel_Component == "fs" and row.Vulnerability_Type == "RC":
        mask.append(True)
    else:
        mask.append(False)
        
maskedExploitCatInfo = allExploitCatInfo[mask]
maskedExploitCatInfo

,EID,Kernel_Component,Vulnerability_Type
88,44302,fs,RC
111,895,fs,RC
129,33321,fs,RC
131,33322,fs,RC
137,40812,fs,RC
145,43345,fs,RC


In [71]:
combine_ngram(list(set(maskedExploitCatInfo.EID.tolist())), "RC_fs")

Finished making RC_fs ngram excel! - total EID #: 6


### IC & fs

In [72]:
# IC 취약점 카테고리와 fs 컴포넌트에 속하는 익스플로잇 모으기
mask = list()
for index, row in allExploitCatInfo.iterrows():
    if row.Kernel_Component == "fs" and row.Vulnerability_Type == "IC":
        mask.append(True)
    else:
        mask.append(False)
        
maskedExploitCatInfo = allExploitCatInfo[mask]
maskedExploitCatInfo

,EID,Kernel_Component,Vulnerability_Type
16,1397,fs,IC
29,19419,fs,IC
41,25202,fs,IC
92,45516,fs,IC
103,5092,fs,IC
106,5093,fs,IC


In [73]:
combine_ngram(list(set(maskedExploitCatInfo.EID.tolist())), "IC_fs")

Finished making IC_fs ngram excel! - total EID #: 6


### PU & net

In [74]:
# PU 취약점 카테고리와 net 컴포넌트에 속하는 익스플로잇 모으기
mask = list()
for index, row in allExploitCatInfo.iterrows():
    if row.Kernel_Component == "net" and row.Vulnerability_Type == "PU":
        mask.append(True)
    else:
        mask.append(False)
        
maskedExploitCatInfo = allExploitCatInfo[mask]
maskedExploitCatInfo

,EID,Kernel_Component,Vulnerability_Type
20,15704,net,PU
74,41458,net,PU
116,9542,net,PU
118,9575,net,PU
142,40871,net,PU
162,47170,net,PU


In [75]:
combine_ngram(list(set(maskedExploitCatInfo.EID.tolist())), "PU_net")

Finished making PU_net ngram excel! - total EID #: 6


### IC & kernel

In [76]:
# IC 취약점 카테고리와 kernel 컴포넌트에 속하는 익스플로잇 모으기
mask = list()
for index, row in allExploitCatInfo.iterrows():
    if row.Kernel_Component == "kernel" and row.Vulnerability_Type == "IC":
        mask.append(True)
    else:
        mask.append(False)
        
maskedExploitCatInfo = allExploitCatInfo[mask]
maskedExploitCatInfo

,EID,Kernel_Component,Vulnerability_Type
44,25444,kernel,IC
46,26131,kernel,IC
58,33589,kernel,IC
150,44298,kernel,IC
156,45010,kernel,IC


In [77]:
combine_ngram(list(set(maskedExploitCatInfo.EID.tolist())), "IC_kernel")

Finished making IC_kernel ngram excel! - total EID #: 5


# 전체 익스플로잇에서 가장 많이 나온 N-gram의 카테고리별 분포 현황
## soft
### Vulnerability Category

In [786]:
max_exploit_cat_ngram = soft_max_exploit_ngram[["N","EIDs"]]
max_exploit_cat_ngram.head()

,N,EIDs
0,2.0,"[10613, 12, 1397, 14814, 15023, 15024, 15285, ..."
1,3.0,"[1397, 14814, 15023, 15024, 15285, 15704, 1577..."
2,4.0,"[1397, 14814, 15023, 15024, 15285, 15704, 1577..."
3,5.0,"[1397, 14814, 15023, 15024, 15285, 15704, 1577..."
4,6.0,"[12, 14814, 15023, 15024, 15285, 15704, 15774,..."


In [787]:
allExploitCatInfo.head()

,EID,Kernel_Component,Vulnerability_Type
0,2013,fs,BA
1,4756,net,BA
2,8678,kernel,RC
3,10613,fs,ETC
4,15023,arch,BA


In [788]:
# max_exploit_cat_ngram = max_exploit_ngram
vulCat = list(set(allExploitCatInfo["Vulnerability_Type"].to_list()))
for v in vulCat:
    max_exploit_cat_ngram[v] = [0 for i in range(len(max_exploit_cat_ngram))]
max_exploit_cat_ngram["ETC"] = [0 for i in range(len(max_exploit_cat_ngram))]
max_exploit_cat_ngram.head()

for index1, row1 in max_exploit_cat_ngram.iterrows():
    for e in row1.EIDs:
        Exist = False
        for index2, row2 in allExploitCatInfo.iterrows():
            if str(e) == str(row2.EID):
                max_exploit_cat_ngram[row2.Vulnerability_Type][index1] += 1
                Exist = True
        if Exist == False:
            max_exploit_cat_ngram["ETC"][index1] += 1
            
max_exploit_cat_ngram.to_excel("max_exploit_cat_ngram.xlsx")
max_exploit_cat_ngram.head()

/tmp/ipykernel_485891/1018171913.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  max_exploit_cat_ngram[v] = [0 for i in range(len(max_exploit_cat_ngram))]
/tmp/ipykernel_485891/1018171913.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  max_exploit_cat_ngram[row2.Vulnerability_Type][index1] += 1


,N,EIDs,IV,IL,AC,IC,PU,ETC,BA,RC
0,2.0,"[10613, 12, 1397, 14814, 15023, 15024, 15285, ...",11,3,19,9,7,1,19,8
1,3.0,"[1397, 14814, 15023, 15024, 15285, 15704, 1577...",8,3,7,5,4,0,11,6
2,4.0,"[1397, 14814, 15023, 15024, 15285, 15704, 1577...",8,3,6,5,4,0,9,5
3,5.0,"[1397, 14814, 15023, 15024, 15285, 15704, 1577...",6,3,6,5,4,0,9,5
4,6.0,"[12, 14814, 15023, 15024, 15285, 15704, 15774,...",6,3,10,4,4,0,11,4


## Soft
### Kernel Component Category

In [102]:
max_exploit_cat_ngram = soft_max_exploit_ngram[["N","EIDs"]]
max_exploit_cat_ngram.head()

,N,EIDs
0,2.0,"[10613, 12, 1397, 14814, 15023, 15024, 15285, ..."
1,3.0,"[1397, 14814, 15023, 15024, 15285, 15704, 1577..."
2,4.0,"[1397, 14814, 15023, 15024, 15285, 15704, 1577..."
3,5.0,"[1397, 14814, 15023, 15024, 15285, 15704, 1577..."
4,6.0,"[12, 14814, 15023, 15024, 15285, 15704, 15774,..."


In [103]:
allExploitCatInfo.head()

,EID,Kernel_Component,Vulnerability_Type
0,2013,fs,BA
1,4756,net,BA
2,8678,kernel,RC
3,10613,fs,ETC
4,15023,arch,BA


In [104]:
# max_exploit_cat_ngram = max_exploit_ngram
vulCat = list(set(allExploitCatInfo["Kernel_Component"].to_list()))
for v in vulCat:
    max_exploit_cat_ngram[v] = [0 for i in range(len(max_exploit_cat_ngram))]
max_exploit_cat_ngram.head()

for index1, row1 in max_exploit_cat_ngram.iterrows():
    for e in row1.EIDs:
        for index2, row2 in allExploitCatInfo.iterrows():
            if str(e) == str(row2.EID):
                max_exploit_cat_ngram[row2.Kernel_Component][index1] += 1
            
max_exploit_cat_ngram.to_excel("max_exploit_cat_ngram(kernel_component)_soft.xlsx")
max_exploit_cat_ngram.head()

/tmp/ipykernel_398327/1815653179.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  max_exploit_cat_ngram[v] = [0 for i in range(len(max_exploit_cat_ngram))]
/tmp/ipykernel_398327/1815653179.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  max_exploit_cat_ngram[row2.Kernel_Component][index1] += 1


,N,EIDs,lib,include,arch,ipc,net,fs,kernel,mm,drivers,security
0,2.0,"[10613, 12, 1397, 14814, 15023, 15024, 15285, ...",2,1,2,2,22,22,11,8,5,2
1,3.0,"[1397, 14814, 15023, 15024, 15285, 15704, 1577...",0,1,1,1,18,6,5,6,5,1
2,4.0,"[1397, 14814, 15023, 15024, 15285, 15704, 1577...",0,1,1,1,18,6,4,4,4,1
3,5.0,"[1397, 14814, 15023, 15024, 15285, 15704, 1577...",0,1,1,1,18,5,3,4,4,1
4,6.0,"[12, 14814, 15023, 15024, 15285, 15704, 15774,...",1,1,1,1,20,4,6,3,4,1


## NEW Hard
### Kernel Component Category

In [96]:
max_exploit_cat_ngram = hard_max_exploit_ngram[["N","EIDs"]]
max_exploit_cat_ngram.head()

,N,EIDs
0,2.0,"[15285, 15774, 35370, 41995, 44303, 43029, 17787]"
1,3.0,"[33516, 37167, 45516, 50808, 895]"
2,4.0,"[895, 20979, 35370, 45553]"
3,5.0,"[43418, 47168, 47169, 6851]"
4,6.0,"[15774, 35370, 40616]"


In [97]:
allExploitCatInfo.head()

,EID,Kernel_Component,Vulnerability_Type
0,2013,fs,BA
1,4756,net,BA
2,8678,kernel,RC
3,10613,fs,ETC
4,15023,arch,BA


In [99]:
# max_exploit_cat_ngram = max_exploit_ngram
vulCat = list(set(allExploitCatInfo["Kernel_Component"].to_list()))
for v in vulCat:
    max_exploit_cat_ngram[v] = [0 for i in range(len(max_exploit_cat_ngram))]
# max_exploit_cat_ngram["ETC"] = [0 for i in range(len(max_exploit_cat_ngram))]
max_exploit_cat_ngram.head()

for index1, row1 in max_exploit_cat_ngram.iterrows():
    for e in row1.EIDs:
        Exist = False
        for index2, row2 in allExploitCatInfo.iterrows():
            if str(e) == str(row2.EID):
                max_exploit_cat_ngram[row2.Kernel_Component][index1] += 1
                Exist = True
        if Exist == False:
            print(row2.EID, "nono")
            
max_exploit_cat_ngram.to_excel("max_exploit_cat_ngram(kernel_component)_hard.xlsx")
max_exploit_cat_ngram.head()

/tmp/ipykernel_398327/1162858900.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  max_exploit_cat_ngram[row2.Kernel_Component][index1] += 1


,N,EIDs,lib,include,arch,ipc,net,fs,kernel,mm,drivers,security
0,2.0,"[15285, 15774, 35370, 41995, 44303, 43029, 17787]",0,0,0,1,4,0,2,2,2,0
1,3.0,"[33516, 37167, 45516, 50808, 895]",0,0,0,0,0,4,0,0,2,0
2,4.0,"[895, 20979, 35370, 45553]",0,0,0,1,0,2,1,0,0,0
3,5.0,"[43418, 47168, 47169, 6851]",0,0,0,0,6,1,0,0,0,0
4,6.0,"[15774, 35370, 40616]",0,0,0,0,0,0,1,2,2,0


## Docker Default Seccomp Profile과의 비교

In [2]:
# Docker Default Seccomp Profile Load
import pandas as pd
docker = pd.read_excel("docker_default_seccomp.xlsx")
docker.head()

,syscall
0,acct
1,add_key
2,bpf
3,clock_adjtime
4,clock_settime


### 시스템콜 이름 -> 번호 전환 함수

In [51]:
import subprocess
# 시스템콜 번호를 시스템콜 이름으로 바꾸기 위함 함수
def convert_name_num(sysname):
    if sysname == "vm86old" or sysname == "vm86":
        return "436"
    if sysname == "umount":
        return "166"
    cmd = f'grep " __NR_{sysname} " /tmp/x86_64.syscall'
    sys_num = subprocess.check_output(cmd, shell=True).decode().strip().split()[2]

    return sys_num

In [4]:
# docker default seccomp 숫자로 바꾸기
for index, row in docker.iterrows():
    if row.syscall == "stime":
        continue
    docker["syscall"][index] = convert_name_num(row.syscall)
docker.head()

,syscall
0,163
1,248
2,321
3,305
4,227


In [8]:
# exploit all path 돌아가며 docker seccomp에 없는 시스템콜 있는 exploit code 찾기
no_docker_seccomp = list()
for eid, pathset in eid_all_path_dict.items():
    BLOCK = False
    for path in pathset:
        for block_syscall in docker.syscall.tolist():
            if block_syscall in path:
                BLOCK = True
    if BLOCK == False:
        no_docker_seccomp.append(eid)
            
no_docker_seccomp = list(set(no_docker_seccomp))
print(len(no_docker_seccomp), no_docker_seccomp)

18 ['5092', '31574', '37722', '6851', '15916', '24555', '32751', '20000', '5093', '20979', '37167', '45516', '9545', '10613', '19419', '15774', '27297', '25202']


In [11]:
# Docker Seccomp Profile로 Block할 수 없는 Exploit Path
no_docker_seccomp_df = pd.DataFrame(columns=["EID","CVEID","syscall_sequence"])
eid_pathset = dict()
for eid, pathset in eid_all_path_dict.items():
    if eid in no_docker_seccomp:
        eid_pathset["EID"] = eid
        eid_pathset["CVEID"] = allExploitInfo["CVEID"][allExploitInfo["EID"]==eid].tolist()
        eid_pathset["syscall_sequence"] = pathset
        no_docker_seccomp_df = no_docker_seccomp_df.append(eid_pathset, ignore_index=True)
no_docker_seccomp_df.head()

,EID,CVEID,syscall_sequence
0,10613,[],"[[102, 12, 12, 59], [102], [102, 12, 12, 59], ..."
1,15774,[2010-4347],"[[12, 12, 257, 5, 0, 5, 0, 5, 0, 3, uname, 12,..."
2,15916,[],"[[prepare_kernel_cred, commit_cred], [prepare_..."
3,19419,[1999-1166],"[[39, 102, 302, 160, 160, 160, 160, 160, 160, ..."
4,20000,[2000-0506],"[[12, 12, 12, 12, capset, 59], [12, 12, 12, 12..."


In [18]:
# excel에 저장
no_docker_seccomp_df.to_excel("docker_defualt_seccomp_cannot_block_list.xlsx")

In [12]:
allExploitInfo["CVEID"][allExploitInfo["EID"]=="5093"].tolist()

['2008-0009']

In [14]:
import numpy as np
# 32751,37167,5093 제외시키기
for index, row in no_docker_seccomp_df.iterrows():
    if row.EID == "32751" or row.EID == "37167" or row.EID == "5093":
        no_docker_seccomp_df["EID"][index] = np.nan
no_docker_seccomp_df = no_docker_seccomp_df.dropna(axis=0)

nan
nan
nan


# Docker Seccomp로 못막는 Exploit이 사용하는 개별 System Call Set 조사

In [17]:
empty_path = [[] for i in range(len(no_docker_seccomp_df))]
no_docker_seccomp_df["syscall_set"] = empty_path
path_set = list()

for index, row in no_docker_seccomp_df.iterrows():
    for path in row.syscall_sequence:
        for syscall in path:
            if syscall.isdigit():
                path_set.append(convert_num_name(syscall))
            else:
                path_set.append(syscall)
    no_docker_seccomp_df["syscall_set"][index] = list(set(path_set))
    path_set = []
no_docker_seccomp_df.head()

,EID,CVEID,syscall_sequence,syscall_set
0,10613,[],"[[102, 12, 12, 59], [102], [102, 12, 12, 59], ...","[pipe, rt_sigaction, munmap, vmsplice, setresu..."
1,15774,[2010-4347],"[[12, 12, 257, 5, 0, 5, 0, 5, 0, 3, uname, 12,...","[strrchr, read, brk, futimesat, get_symbol, st..."
2,15916,[],"[[prepare_kernel_cred, commit_cred], [prepare_...","[strrchr, read, brk, socket, mmap, get_kernel_..."
3,19419,[1999-1166],"[[39, 102, 302, 160, 160, 160, 160, 160, 160, ...","[setgid, setrlimit, setgroups, prlimit64, exec..."
4,20000,[2000-0506],"[[12, 12, 12, 12, capset, 59], [12, 12, 12, 12...","[ex_main, add_main, setresuid, brk, execve, ca..."


In [19]:
no_docker_seccomp_df[["EID","syscall_set"]].to_excel("docker_cannot_block_list_syscall_set.xlsx")

### 전체 다 합친 System call set

In [20]:
no_docker_total_syscall_set = list()
for index, row in no_docker_seccomp_df.iterrows():
    no_docker_total_syscall_set.extend(row.syscall_set)
no_docker_total_syscall_set = list(set(no_docker_total_syscall_set))
print(no_docker_total_syscall_set)
no_docker_total_syscall_set_only_syscall =list()
# 시스템콜 인것만 거르기
for syscall in no_docker_total_syscall_set:
    try:
        cmd = f'grep "__NR_{syscall} " /tmp/x86_64.syscall'
        sys_name = subprocess.check_output(cmd, shell=True).decode()
        no_docker_total_syscall_set_only_syscall.append(syscall)
    except:
        print(f"no syscall - {syscall}")
        continue
print(no_docker_total_syscall_set_only_syscall)
print(len(no_docker_total_syscall_set_only_syscall))

['pipe', 'epoll_ctl', 'sched_setscheduler', 'alloca', 'siglongjmp', 'setrlimit', 'mmap', 'getpid', 'getuid', 'sendto', 'close', 'spew_main', 'setreuid', 'setuid', 'fstat', 'socketpair', 'add_main', 'modify_ldt', 'exploit', 'getgid', 'futimesat', 'get_symbol', 'get_kernel_sym', 'epoll_wait', 'unlink', 'uname', 'set_ldt', 'mlock', 'sigaltstack', '.ABNORMAL_DISPATCHER', 'read', 'sethandler', 'ftruncate', 'setresuid', 'setresgid', 'socket', 'prlimit64', 'sched_setaffinity', 'prepare_kernel_cred', 'openat', '__assert_fail', 'rt_sigaction', 'strrchr', 'munmap', 'ex_main', 'epoll_create', 'vmsplice', 'setgid', 'dup2', 'brk', 'setgroups', 'capget', 'stat', 'ioctl', 'execve', 'commit_cred', 'capset', 'vuln_prog_main', 'clock_nanosleep', 'sendfile', 'lseek', 'write']
no syscall - alloca
no syscall - siglongjmp
no syscall - spew_main
no syscall - add_main
no syscall - exploit
no syscall - get_symbol
no syscall - get_kernel_sym
no syscall - set_ldt
no syscall - .ABNORMAL_DISPATCHER
no syscall - se

# Docker seccomp non-mitigate exploit의 coverage

In [21]:
soft_dedup_ngramdf.head()

,N,Ngram_syscall,num_of_exploit,EIDs
0,66,"[14, 248, 248, 248, 41, 248, 248, 248, 54, 0, ...",2,"[40871, 47170]"
156,49,"[230, 0, 3, 5, 9, 10, 12, 13, 14, 17, 56, 218,...",2,"[40871, 47170]"
251,45,"[12, 12, 5, 0, 257, 4, 12, 12, 5, 0, 257, 4, 1...",2,"[47163, 50541]"
269,44,"[102, 104, 272, 272, 257, 1, 3, 257, 1, 3, 257...",2,"[43418, 47169]"
275,44,"[12, 12, 257, 12, 12, 257, 5, 0, 5, 0, 5, 0, 5...",2,"[14814, 15023]"


In [22]:
no_docker_seccomp_df.head()

,EID,CVEID,syscall_sequence,syscall_set
0,10613,[],"[[102, 12, 12, 59], [102], [102, 12, 12, 59], ...","[pipe, rt_sigaction, munmap, vmsplice, setresu..."
1,15774,[2010-4347],"[[12, 12, 257, 5, 0, 5, 0, 5, 0, 3, uname, 12,...","[strrchr, read, brk, futimesat, get_symbol, st..."
2,15916,[],"[[prepare_kernel_cred, commit_cred], [prepare_...","[strrchr, read, brk, socket, mmap, get_kernel_..."
3,19419,[1999-1166],"[[39, 102, 302, 160, 160, 160, 160, 160, 160, ...","[setgid, setrlimit, setgroups, prlimit64, exec..."
4,20000,[2000-0506],"[[12, 12, 12, 12, capset, 59], [12, 12, 12, 12...","[ex_main, add_main, setresuid, brk, execve, ca..."


In [23]:
mitigate = [False for i in range(len(no_docker_seccomp_df))]
no_docker_seccomp_df["mitigate"] = mitigate
mitigate_N = [0 for i in range(len(no_docker_seccomp_df))]
no_docker_seccomp_df["mitigate_N"] = mitigate_N
mitigate_ngram = [[] for i in range(len(no_docker_seccomp_df))]
no_docker_seccomp_df["mitigate_ngram"] = mitigate_ngram

Ngram_syscall_sysname = list()
for index1, row1 in no_docker_seccomp_df.iterrows():
    for index2, row2 in soft_dedup_ngramdf.iterrows():
        if row1.EID in row2.EIDs:
            no_docker_seccomp_df["mitigate"][index1] = True
            no_docker_seccomp_df["mitigate_N"][index1] = row2.N
            for syscall in row2.Ngram_syscall:
                Ngram_syscall_sysname.append(convert_num_name(syscall))
            no_docker_seccomp_df["mitigate_ngram"][index1] = Ngram_syscall_sysname
            Ngram_syscall_sysname = []
            break
no_docker_seccomp_df.head()

/tmp/ipykernel_3838180/3648654127.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_docker_seccomp_df["mitigate"][index1] = True
/tmp/ipykernel_3838180/3648654127.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_docker_seccomp_df["mitigate_N"][index1] = row2.N
/tmp/ipykernel_3838180/3648654127.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_docker_seccomp_df["mitigate_ngram"][index1] = Ngram_syscall_sysname


,EID,CVEID,syscall_sequence,syscall_set,mitigate,mitigate_N,mitigate_ngram
0,10613,[],"[[102, 12, 12, 59], [102], [102, 12, 12, 59], ...","[pipe, rt_sigaction, munmap, vmsplice, setresu...",True,13,"[getuid, getgid, setresuid, setresgid, mmap, m..."
1,15774,[2010-4347],"[[12, 12, 257, 5, 0, 5, 0, 5, 0, 3, uname, 12,...","[strrchr, read, brk, futimesat, get_symbol, st...",True,26,"[brk, brk, openat, brk, brk, openat, fstat, re..."
2,15916,[],"[[prepare_kernel_cred, commit_cred], [prepare_...","[strrchr, read, brk, socket, mmap, get_kernel_...",True,22,"[brk, brk, openat, brk, brk, openat, fstat, re..."
3,19419,[1999-1166],"[[39, 102, 302, 160, 160, 160, 160, 160, 160, ...","[setgid, setrlimit, setgroups, prlimit64, exec...",True,2,"[setuid, setgid]"
4,20000,[2000-0506],"[[12, 12, 12, 12, capset, 59], [12, 12, 12, 12...","[ex_main, add_main, setresuid, brk, execve, ca...",True,6,"[openat, write, close, openat, write, close]"


In [24]:
# excel로 저장
no_docker_seccomp_df.to_excel("docker_cannot_block_list_mitigation_ngram.xlsx")


# Docker Seccomp Non-mitigate Exploit의 패턴 분석

In [ ]:
allExploitInfo = pd.read_pickle('allExploitInfo.pkl')

In [37]:
soft_dedup_ngramdf.head()

,N,Ngram_syscall,num_of_exploit,EIDs
0,66,"[14, 248, 248, 248, 41, 248, 248, 248, 54, 0, ...",2,"[40871, 47170]"
156,49,"[230, 0, 3, 5, 9, 10, 12, 13, 14, 17, 56, 218,...",2,"[40871, 47170]"
251,45,"[12, 12, 5, 0, 257, 4, 12, 12, 5, 0, 257, 4, 1...",2,"[47163, 50541]"
269,44,"[102, 104, 272, 272, 257, 1, 3, 257, 1, 3, 257...",2,"[43418, 47169]"
275,44,"[12, 12, 257, 12, 12, 257, 5, 0, 5, 0, 5, 0, 5...",2,"[14814, 15023]"


In [63]:
# docker default seccomp로 mitigate할 수 없는 exploit EDB-ID list
no_docker_seccomp_pattern_eids = no_docker_seccomp_df[["EID"]]
no_docker_seccomp_pattern_eids = no_docker_seccomp_pattern_eids["EID"].to_list()

# 패턴 분석을 위한 데이터 프레임 생성
no_docker_seccomp_pattern_df = pd.DataFrame(columns=["EID", "N", "Ngram_pattern","block_EIDs","block_CVEs"])
no_docker_seccomp_pattern_dict = {"EID":"", "N":0, "Ngram_pattern":[],"block_EIDs":[],"block_CVEs":[]}

Ngram_sysname = list()
for no_seccomp_eid in no_docker_seccomp_pattern_eids:
    for index, row in soft_dedup_ngramdf.iterrows():
        no_docker_seccomp_pattern_dict["EID"] = no_seccomp_eid
        if no_seccomp_eid in row.EIDs:
            no_docker_seccomp_pattern_dict["N"] = row.N
            for syscall in row.Ngram_syscall:
                Ngram_sysname.append(convert_num_name(syscall))
            no_docker_seccomp_pattern_dict["Ngram_pattern"] = Ngram_sysname
            block_EIDs = row.EIDs.copy()
            block_EIDs.remove(no_seccomp_eid)
            no_docker_seccomp_pattern_dict["block_EIDs"] = block_EIDs
            no_docker_seccomp_pattern_df = no_docker_seccomp_pattern_df.append(no_docker_seccomp_pattern_dict, ignore_index=True)
            no_docker_seccomp_pattern_dict = {"EID":"", "N":0, "Ngram_pattern":[],"block_EIDs":[],"block_CVEs":[]}
            Ngram_sysname = []
            
no_docker_seccomp_pattern_df.head()

,EID,N,Ngram_pattern,block_EIDs,block_CVEs
0,10613,13,"[getuid, getgid, setresuid, setresgid, mmap, m...",[5092],[]
1,10613,4,"[mmap, mmap, mmap, mmap]","[3, 35370, 40871, 45516, 47170, 50541, 5092, 895]",[]
2,10613,4,"[getuid, getgid, setresuid, setresgid]","[33321, 33322, 5092, 5093]",[]
3,10613,4,"[getuid, brk, brk, execve]","[5092, 5093]",[]
4,10613,3,"[mmap, mmap, mmap]","[17787, 3, 35370, 40871, 45516, 45553, 47170, ...",[]


In [64]:
# block CVE field 채우기
# EID에 해당하는 CVE 먼저 채우기
for index, row in no_docker_seccomp_pattern_df.iterrows():
    CVE = allExploitInfo["CVEID"][allExploitInfo["EID"] == row.EID].to_list()
    if CVE:
        no_docker_seccomp_pattern_df["block_CVEs"][index].extend(CVE)
        
# block EIDs에 해당하는 CVE 마저 채우기
for index, row in no_docker_seccomp_pattern_df.iterrows():
    cves = []
    for eid in row.block_EIDs:
        cves.extend(allExploitInfo["CVEID"][allExploitInfo["EID"]==eid].tolist())
    no_docker_seccomp_pattern_df["block_CVEs"][index].extend(cves)
    cves = []
    # 중복 제거
    no_docker_seccomp_pattern_df["block_CVEs"][index] = list(set(no_docker_seccomp_pattern_df["block_CVEs"][index]))

no_docker_seccomp_pattern_df.tail()

,EID,N,Ngram_pattern,block_EIDs,block_CVEs
207,6851,2,"[openat, stat]","[15774, 2006, 2031, 22362, 3, 37167, 38390, 39...","[2019-13272, 2009-1185, 2006-2451, 2010-4347, ..."
208,9545,3,"[getuid, getgid, mmap]",[25444],"[2009-2692, 2013-2094]"
209,9545,2,"[socket, unlink]",[18411],"[2009-2692, 2012-0056]"
210,9545,2,"[mmap, socket]","[15916, 15944, 926, 9542]","[2009-2692, 2009-2698, 2005-1294, 2005-0750]"
211,9545,2,"[getuid, getgid]","[10613, 14814, 17391, 24555, 25444, 33321, 333...","[2010-2959, 2009-1046, 2009-3547, 2008-0600, 2..."


In [65]:
# excel에 저장
no_docker_seccomp_pattern_df.to_excel("no_docker_seccomp_pattern.xlsx")

## 총 패턴 통계 내기

In [78]:
# docker default seccomp로 mitigate할 수 없는 exploit EDB-ID list
no_docker_seccomp_pattern_eids = no_docker_seccomp_df[["EID"]]
no_docker_seccomp_pattern_eids = no_docker_seccomp_pattern_eids["EID"].to_list()

# 패턴 분석 통계를 위한 데이터 프레임 생성
no_docker_seccomp_pattern_total_df = pd.DataFrame(columns=["EID", "total_Ngram_pattern","total_block_EIDs","total_block_CVEs"])
no_docker_seccomp_pattern_total_dict = {"EID":"", "total_Ngram_pattern":0,"total_block_EIDs":0,"total_block_CVEs":0}
block_EIDs, block_CVEs = [], []

for index, row in no_docker_seccomp_pattern_df.iterrows():
    if no_docker_seccomp_pattern_total_dict["EID"] == "":
        no_docker_seccomp_pattern_total_dict["EID"] = row.EID
    elif not no_docker_seccomp_pattern_total_dict["EID"] == row.EID:
        # append
        no_docker_seccomp_pattern_total_dict["total_block_EIDs"] = len(set(block_EIDs))
        no_docker_seccomp_pattern_total_dict["total_block_CVEs"] = len(set(block_CVEs))
        no_docker_seccomp_pattern_total_df = no_docker_seccomp_pattern_total_df.append(no_docker_seccomp_pattern_total_dict, ignore_index = True)
        no_docker_seccomp_pattern_total_dict = {"EID":"", "total_Ngram_pattern":0,"total_block_EIDs":0,"total_block_CVEs":0}
        #new eid
        no_docker_seccomp_pattern_total_dict["EID"] = row.EID
        block_EIDs, block_CVEs = [], []
    no_docker_seccomp_pattern_total_dict["total_Ngram_pattern"] += 1
    block_EIDs.extend(row.block_EIDs)
    block_CVEs.extend(row.block_CVEs)

# excel에 저장
no_docker_seccomp_pattern_total_df.to_excel("no_docker_seccomp_pattern_total.xlsx")
no_docker_seccomp_pattern_total_df.head()

,EID,total_Ngram_pattern,total_block_EIDs,total_block_CVEs
0,10613,15,69,49
1,15774,47,73,53
2,15916,36,70,51
3,19419,1,1,2
4,20000,8,63,49


# Docker Seccomp Profile의 Exploit Coverage 조사

In [109]:
for EID, pathset in eid_all_path_dict.items():
    for path in pathset:
        if "__NR_vm86old" in path:
            print(EID, path)

5093 ['102', '104', '117', '119', 'get_target', '22', '1', '278', '__NR_vm86old', '102', '12', '12', '59']
5093 ['102', '104', '117', '119', 'get_target', '22', '1', '278', '__NR_vm86old', '102']
5093 ['102', '104', '117', '119', 'get_target', '22', '1', '278', '__NR_vm86old', '102', '12', '12', '59']
5093 ['102', '104', '117', '119', 'get_target', '22', '1', '278', '__NR_vm86old', '102']


In [127]:
docker = pd.read_excel("docker_default_seccomp.xlsx")
# docker default seccomp 숫자로 바꾸기
for index, row in docker.iterrows():
    if row.syscall == "stime":
        continue
    docker["syscall"][index] = convert_name_num(row.syscall)
docker.head()

,syscall
0,163
1,248
2,321
3,305
4,227


In [140]:
docker_exploit_coverage_df = pd.DataFrame()
docker_exploit_coverage_df = docker.copy()
docker_exploit_coverage_df["EIDs"] = [[] for i in range(len(docker))]
docker_exploit_coverage_df["E_num"] = [0 for i in range(len(docker))]
for index, row in docker_exploit_coverage_df.iterrows():
    if row.syscall == "436":
        docker_exploit_coverage_df["syscall"][index] = "__NR_vm86old"
    if row.syscall == "stime":
        docker_exploit_coverage_df["syscall"][index] = "stime"
    if row.syscall == "308":
        docker_exploit_coverage_df["syscall"][index] = "setns"
    print(docker_exploit_coverage_df["syscall"][index], type(docker_exploit_coverage_df["syscall"][index]))
docker_exploit_coverage_df.head()

163 <class 'str'>
248 <class 'str'>
321 <class 'str'>
305 <class 'str'>
227 <class 'str'>
56 <class 'str'>
174 <class 'str'>
176 <class 'str'>
313 <class 'str'>
177 <class 'str'>
239 <class 'str'>
175 <class 'str'>
173 <class 'str'>
172 <class 'str'>
312 <class 'str'>
320 <class 'str'>
246 <class 'str'>
250 <class 'str'>
212 <class 'str'>
237 <class 'str'>
165 <class 'str'>
279 <class 'str'>
303 <class 'str'>
180 <class 'str'>
304 <class 'str'>
298 <class 'str'>
135 <class 'str'>
155 <class 'str'>
310 <class 'str'>
311 <class 'str'>
101 <class 'str'>
178 <class 'str'>
179 <class 'str'>
169 <class 'str'>
249 <class 'str'>
238 <class 'str'>
setns <class 'str'>
164 <class 'str'>
stime <class 'str'>
167 <class 'str'>
168 <class 'str'>
139 <class 'str'>
156 <class 'str'>
166 <class 'str'>
166 <class 'str'>
272 <class 'str'>
134 <class 'str'>
323 <class 'str'>
136 <class 'str'>
__NR_vm86old <class 'str'>
__NR_vm86old <class 'str'>


/tmp/ipykernel_3838180/995810731.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  docker_exploit_coverage_df["syscall"][index] = "setns"
/tmp/ipykernel_3838180/995810731.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  docker_exploit_coverage_df["syscall"][index] = "stime"
/tmp/ipykernel_3838180/995810731.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  docker_exploit_coverage_df["syscall"][index] = "__NR_vm86old"


,syscall,EIDs,E_num
0,163,[],0
1,248,[],0
2,321,[],0
3,305,[],0
4,227,[],0


In [141]:
for index, row in docker_exploit_coverage_df.iterrows():
    for EID, pathSet in eid_all_path_dict.items():
        for seq in pathSet:
            if row.syscall in seq:
                docker_exploit_coverage_df["EIDs"][index].append(EID)
for index, row in docker_exploit_coverage_df.iterrows():
    docker_exploit_coverage_df["E_num"][index] = len(set(row.EIDs))
    docker_exploit_coverage_df["EIDs"][index] = set(row.EIDs)
docker_exploit_coverage_df.head()

/tmp/ipykernel_3838180/1710129678.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  docker_exploit_coverage_df["E_num"][index] = len(set(row.EIDs))
/tmp/ipykernel_3838180/1710129678.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  docker_exploit_coverage_df["EIDs"][index] = set(row.EIDs)


,syscall,EIDs,E_num
0,163,{},0
1,248,"{40871, 47170}",2
2,321,"{45010, 44298}",2
3,305,{},0
4,227,{},0


In [142]:
for index, row in docker_exploit_coverage_df.iterrows():
    if row.syscall == "stime":
        docker_exploit_coverage_df["syscall"][index] = "stime"
    elif row.syscall == "__NR_vm86old":
        docker_exploit_coverage_df["syscall"][index] = "vm86old"
    elif row.syscall == "setns":
        docker_exploit_coverage_df["syscall"][index] = "setns"
    else:
        docker_exploit_coverage_df["syscall"][index] = convert_num_name(row.syscall)
    
docker_exploit_coverage_df.head()

/tmp/ipykernel_3838180/1310071248.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  docker_exploit_coverage_df["syscall"][index] = convert_num_name(row.syscall)
/tmp/ipykernel_3838180/1310071248.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  docker_exploit_coverage_df["syscall"][index] = "setns"
/tmp/ipykernel_3838180/1310071248.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  docker_exploit_coverage_df["syscall"][index] = "s

,syscall,EIDs,E_num
0,acct,{},0
1,add_key,"{40871, 47170}",2
2,bpf,"{45010, 44298}",2
3,clock_adjtime,{},0
4,clock_settime,{},0


In [143]:
# excel에 저장
docker_exploit_coverage_df.to_excel("docker_exploit_coverage.xlsx")

# Docker의 Capability까지 고려한 Mitigable List에 대한 분석

In [159]:
import pandas as pd

# Docker Seccomp Profile로 완화할 수 있는 Exploit EDB-iD
mitigable_EID = pd.read_excel("docker_mitigable_exploit_EDBID.xlsx")
print(len(mitigable_EID))
mitigable_EID.head()

45


,EID
0,3
1,12
2,895
3,4460
4,5093


In [157]:
# Docker Seccomp Profile로 완화할 수 없는 Exploit
non_mitigable_df = pd.DataFrame(columns=["EID","path_set","pattern_set"])
non_mitigable_dict = {"EID":"", "path_set":[], "pattern_set":[]}

mitigable_eid_list = mitigable_EID.EID.to_list()
for EID, path_set in eid_all_path_dict.items():
    if int(EID) in mitigable_eid_list:
        continue
    non_mitigable_dict["EID"] = EID
    non_mitigable_dict["path_set"] = path_set
    non_mitigable_df = non_mitigable_df.append(non_mitigable_dict, ignore_index=True)
    non_mitigable_dict = {"EID":"", "path_set":[], "pattern_set":[]}
print(len(non_mitigable_df))
non_mitigable_df.head()

61


,EID,path_set,pattern_set
0,10613,"[[102, 12, 12, 59], [102], [102, 12, 12, 59], ...",[]
1,1397,"[[125], [125], [125], [125, 125, 125, 125], [1...",[]
2,14814,"[[prepare_kernel_cred, commit_cred], [prepare_...",[]
3,15024,"[[12, 12, 257, 3], [12, 12, 257, 3], [uname], ...",[]
4,15704,"[[12, 12, 257, 5, 0, 5, 0, 5, 0, 5, 0, 5, 0, 5...",[]


# Docker Seccomp로 완화할 수 없는 익스플로잇에 사용된 Syscall Set

In [163]:
non_mitigable_syscall_set_raw = set()
for index, row in non_mitigable_df.iterrows():
    for path in row.path_set:
        for syscall in path:
            non_mitigable_syscall_set_raw.add(syscall)

print(non_mitigable_syscall_set_raw)
print(len(non_mitigable_syscall_set_raw))

{'build_msg_msg', 'my_asprintf', 'kmalloc_pad', '24)', '272', '257', '33', '217', '22', 'get_shell', 'fillsocketcallAT', 'spew_main', '137', 'do_child_upper', 'y0y0stack', '16', '247', '283', '1', '261', '154', 'uname', 'rey0y0code', 'time', '.ABNORMAL_DISPATCHER', '80', 'get_elf_info', 'recv_fd', 'rand', 'start_listener', 'check_procs', 'get_kmalloc_fd', '2', 'get_socklen', 'brute_socket_create', '105', 'kernel_write', '230', 'init_reallocation', 'bindcpu', 'null_poiter_exploit_main', '108', 'strrchr', 'connect_peer', '42', 'ex_main', '110', '__NR_socketcall', 'write_msg', 'htons', '0', '13', '7', '157', 'pthread_kill', '104', '294', 'hax', '61', 'do_mmap', 'rawmemchr', 'getidt', '286', '28', 'get_config', '89', 'get_adjacent_kstacks', '83', '92', 'exploit', '50', '32', 'saveme', 'clr', 'check_zombie_child', '<<<Unknowntree:integer_cst>>>', 'alloc_tioclinux', '119', 'get_total_object', 'read_msg', 'idt_smash', '160', 'bucket=<<<Unknowntree:integer_cst>>>', '107', 'trigger', '59', 'pre

In [164]:
non_mitigable_syscall_set = non_mitigable_syscall_set_raw.copy()
for syscall in non_mitigable_syscall_set_raw:
    if "setns" == syscall:
        non_mitigable_syscall_set.remove("setns")
        non_mitigable_syscall_set.add("308")
    if "time" == syscall:
        non_mitigable_syscall_set.remove("time")
        non_mitigable_syscall_set.add("201")
    if "capset" == syscall:
        non_mitigable_syscall_set.remove("capset")
        non_mitigable_syscall_set.add("126")
    if "fstat" == syscall:
        non_mitigable_syscall_set.remove("fstat")
        non_mitigable_syscall_set.add("5")
    if "uname" == syscall:
        non_mitigable_syscall_set.remove("uname")
        non_mitigable_syscall_set.add("63")
    if "__NR_vm86old" == syscall:
        non_mitigable_syscall_set.remove("__NR_vm86old")
        non_mitigable_syscall_set.add("x113")
    if "sigaction" == syscall:
        non_mitigable_syscall_set.remove("sigaction")
        non_mitigable_syscall_set.add("13")
    if "ioctl" == syscall:
        non_mitigable_syscall_set.remove("ioctl")
        non_mitigable_syscall_set.add("16")
    if "kmalloc" == syscall:
        non_mitigable_syscall_set.remove("kmalloc")
        non_mitigable_syscall_set.add("12")
    if "shutdown" == syscall:
        non_mitigable_syscall_set.remove("shutdown")
        non_mitigable_syscall_set.add("48")
    if "msgrcv" == syscall:
        non_mitigable_syscall_set.remove("msgrcv")
        non_mitigable_syscall_set.add("70")
    if "__NR_socketcall" == syscall:
        non_mitigable_syscall_set.remove("__NR_socketcall")
        non_mitigable_syscall_set.add("x102")
len(non_mitigable_syscall_set)

In [172]:
import subprocess
# 시스템콜 번호를 시스템콜 이름으로 바꾸기 위함 함수
def convert_num_name_new(sysnum):
    if sysnum == "x113":
        return "vm86old"
    if sysnum == "x102":
        return "socketcall"
    cmd = f'grep " {sysnum}$" /tmp/x86_64.syscall'
    sys_name = subprocess.check_output(cmd, shell=True).decode().strip().split()[1].replace("__NR_","")
    return sys_name

In [176]:
non_mitigable_syscall_set_only_syscall = non_mitigable_syscall_set.copy()
for syscall in non_mitigable_syscall_set:
    if syscall == "x113" or syscall == "x102":
        continue
    if syscall == "1073742361":
        non_mitigable_syscall_set_only_syscall.remove(syscall)
    if not syscall.isdigit():
        non_mitigable_syscall_set_only_syscall.remove(syscall)

non_mitigable_syscall_set_only_syscall_sysname = set()
for syscall in non_mitigable_syscall_set_only_syscall:
    non_mitigable_syscall_set_only_syscall_sysname.add(convert_num_name_new(syscall))
print(non_mitigable_syscall_set_only_syscall_sysname)
print(len(non_mitigable_syscall_set_only_syscall_sysname))

{'fcntl', 'sched_yield', 'recvmsg', 'statfs', 'setrlimit', 'fstat', 'getppid', 'mkdir', 'unlink', 'chdir', 'uname', 'time', 'madvise', 'mlock', 'ftruncate', 'inotify_add_watch', 'gettid', 'setsockopt', 'openat', 'setpriority', 'vmsplice', 'brk', 'msgctl', 'rename', 'clock_nanosleep', 'lseek', 'pipe', 'timerfd_create', 'epoll_ctl', 'getpriority', 'accept', 'sendto', 'setreuid', 'waitid', 'setuid', 'sendmmsg', 'wait4', 'arch_prctl', 'modify_ldt', 'set_tid_address', 'getgid', 'futimesat', 'epoll_wait', 'flock', 'sigaltstack', 'setresgid', 'setresuid', 'socket', 'prlimit64', 'chmod', 'rt_sigreturn', 'stat', 'readlink', 'getsockname', 'kill', 'write', 'fchmod', 'getegid', 'alarm', 'socketcall', 'getuid', 'shmget', 'close', 'listen', 'bind', 'msgrcv', 'set_robust_list', 'timerfd_settime', 'setsid', 'sched_setaffinity', 'msgsnd', 'rt_sigaction', 'epoll_create', 'poll', 'setgid', 'pread64', 'ioctl', 'sendfile', 'sched_setscheduler', 'open', 'syslog', 'getcwd', 'mmap', 'getpid', 'getsockopt', '

# 61 Non-mitigable exploit Pattern 조사

In [180]:
non_mitigable_df["mitigate_N"] = [0 for i in range(len(non_mitigable_df))]
non_mitigable_df["mitigate_ngram"] = [0 for i in range(len(non_mitigable_df))]
        
Ngram_syscall_sysname = list()
for index1, row1 in non_mitigable_df.iterrows():
    for index2, row2 in soft_dedup_ngramdf.iterrows():
        if row1.EID in row2.EIDs:
            non_mitigable_df["mitigate_N"][index1] = row2.N
            for syscall in row2.Ngram_syscall:
                Ngram_syscall_sysname.append(convert_num_name(syscall))
            non_mitigable_df["mitigate_ngram"][index1] = Ngram_syscall_sysname
            Ngram_syscall_sysname = []
            break
non_mitigable_df.head()

/tmp/ipykernel_3838180/899098796.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_mitigable_df["mitigate_N"][index1] = row2.N
/tmp/ipykernel_3838180/899098796.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_mitigable_df["mitigate_ngram"][index1] = Ngram_syscall_sysname
/home/mini/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(i

,EID,path_set,pattern_set,mitigate_N,mitigate_ngram
0,10613,"[[102, 12, 12, 59], [102], [102, 12, 12, 59], ...",[],13,"[getuid, getgid, setresuid, setresgid, mmap, m..."
1,1397,"[[125], [125], [125], [125, 125, 125, 125], [1...",[],34,"[epoll_create, socketpair, sendto, socketpair,..."
2,14814,"[[prepare_kernel_cred, commit_cred], [prepare_...",[],44,"[brk, brk, openat, brk, brk, openat, fstat, re..."
3,15024,"[[12, 12, 257, 3], [12, 12, 257, 3], [uname], ...",[],12,"[read, close, uname, brk, brk, openat, fstat, ..."
4,15704,"[[12, 12, 257, 5, 0, 5, 0, 5, 0, 5, 0, 5, 0, 5...",[],26,"[brk, brk, openat, brk, brk, openat, fstat, re..."


In [181]:
non_mitigable_df.to_excel("non_mitigable_exploit_pattern.xlsx")

482

# Glibc가 사용하는 syscall sequence 조사

In [13]:
# glibc test case 불러오기
import subprocess

# 총 익스플로잇에서 사용하는 glibc 종류는 몇 개?
glibcResultFile_total = subprocess.check_output(f"find /opt/output/temp/testcase/*-*-*.c", shell=True).decode().strip().split()
glibcResultFileName_total = list(map(lambda x: x.replace("/opt/output/temp/testcase/","").replace(".c",""), glibcResultFile_total))
glibcResultFileName_total = list(map(lambda x: x.split("-")[0], glibcResultFileName_total))
glibcResultFileName_total = list(set(glibcResultFileName_total))
print("total libc used in exploits: ", len(glibcResultFile_total)+19, len(glibcResultFileName_total)+19)

# 그 중 테스트 성공한 것은 몇 개? (225 종류, 1231개 Case 성공)
glibcResultFile_test = subprocess.check_output(f"find /opt/output/temp/testcase/result/*.txt", shell=True).decode().strip().split()
glibcResultFileName_test = list(map(lambda x: x.replace("/opt/output/temp/testcase/result/","").replace(".txt",""), glibcResultFile_test))
glibcResultFileName_test = list(map(lambda x: x.split("-")[0], glibcResultFileName_test))
glibcResultFileName_test = list(set(glibcResultFileName_test))
print("total libc passed the test: ", len(glibcResultFile_test), len(glibcResultFileName_test))

total libc used in exploits:  4571 255
total libc passed the test:  1231 255


In [14]:
import pandas as pd
# glibc가 호출하는 시스템콜은 각각 몇개?
glibc_syscall_df = pd.DataFrame(columns = ["glibc","syscall_num","syscall_sequence"])
glibc_syscall_df["glibc"] = glibcResultFileName_test
glibc_syscall_df["syscall_num"] = [0 for i in range(len(glibcResultFileName_test))]
glibc_syscall_df["syscall_sequence"] = [[] for i in range(len(glibcResultFileName_test))]
glibc_syscall_df.head()

,glibc,syscall_num,syscall_sequence
0,fchmod,0,[]
1,getchar,0,[]
2,waitpid,0,[]
3,pthread_sigmask,0,[]
4,msync,0,[]


In [15]:
import numpy as np
# defualt 파일 기준으로 count 해보자!
for index, row in glibc_syscall_df.iterrows():
    try:
        glibc_cat_result = subprocess.check_output(f"cat /opt/output/temp/testcase/result/{row.glibc}-default-default.txt", shell=True).decode().strip().split()
        glibc_syscall_df["syscall_num"][index] = len(glibc_cat_result)
        glibc_syscall_df["syscall_sequence"][index] = glibc_cat_result
    except:
        glibc_syscall_df["syscall_num"][index] = np.nan
        glibc_syscall_df["syscall_sequence"][index] = np.nan
glibc_syscall_df.head()

/tmp/ipykernel_1131056/330061725.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  glibc_syscall_df["syscall_num"][index] = len(glibc_cat_result)
/tmp/ipykernel_1131056/330061725.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  glibc_syscall_df["syscall_sequence"][index] = glibc_cat_result
cat: /opt/output/temp/testcase/result/_exit-default-default.txt/tmp/ipykernel_1131056/330061725.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-co

,glibc,syscall_num,syscall_sequence
0,fchmod,1.0,[91]
1,getchar,4.0,"[5, 12, 12, 0]"
2,waitpid,1.0,[61]
3,pthread_sigmask,1.0,[14]
4,msync,1.0,[26]


In [16]:
glibc_syscall_df.to_excel("glibc_syscall.xlsx")

## 시퀀스 길이 10 이상인 libc 함수 각각 몇 개의 Exploit에 출현?

###  그 전에 먼저 실제로 코드에 나온 glibc 인지 검증

In [129]:
sorted_glibc_syscall_df = glibc_syscall_df.sort_values(by=["syscall_num"], ascending=False, ignore_index=True)
sorted_glibc_syscall_df.head()

,glibc,syscall_num,syscall_sequence
0,initgroups,189.0,"[257, 0, 3, 12, 12, 41, 42, 3, 41, 42, 3, 257,..."
1,getpwuid,33.0,"[12, 12, 41, 42, 3, 41, 42, 3, 257, 5, 0, 0, 3..."
2,pclose,23.0,"[61, 13, 59, 9, 56, 10, 17, 12, 11, 3, 257, 5,..."
3,getaddrinfo,22.0,"[9, 257, 3, 0, 10, 5, 11, 41, 42, 307, 47, 7, ..."
4,gethostbyname,18.0,"[9, 10, 257, 3, 0, 5, 11, 42, 44, 41, 12, 4, 7..."


In [130]:
sorted_glibc_syscall_df["codes"] = [[] for i in range(len(sorted_glibc_syscall_df))]
sorted_glibc_syscall_df["originals"] = [[] for i in range(len(sorted_glibc_syscall_df))]
sorted_glibc_syscall_df["test_result"] = [[] for i in range(len(sorted_glibc_syscall_df))]
sorted_glibc_syscall_df.head()

,glibc,syscall_num,syscall_sequence,codes,originals,test_result
0,initgroups,189.0,"[257, 0, 3, 12, 12, 41, 42, 3, 41, 42, 3, 257,...",[],[],[]
1,getpwuid,33.0,"[12, 12, 41, 42, 3, 41, 42, 3, 257, 5, 0, 0, 3...",[],[],[]
2,pclose,23.0,"[61, 13, 59, 9, 56, 10, 17, 12, 11, 3, 257, 5,...",[],[],[]
3,getaddrinfo,22.0,"[9, 257, 3, 0, 10, 5, 11, 41, 42, 307, 47, 7, ...",[],[],[]
4,gethostbyname,18.0,"[9, 10, 257, 3, 0, 5, 11, 42, 44, 41, 12, 4, 7...",[],[],[]


In [131]:
import subprocess
for index, row in sorted_glibc_syscall_df.iterrows():
    try:
        # search in c codes
        cmd = f'grep "{row.glibc}" ../exploit/exploit-db/*.c '
        ret = subprocess.check_output(cmd, shell=True).decode().split("\n")
        ret.pop(-1)
        ret = list(map(lambda x: x.split(":")[0].replace("../exploit/exploit-db/","").replace(".c",""), ret))
        sorted_glibc_syscall_df["codes"][index].extend(ret)
    except:
        print(f"no {row.glibc} in c codes")
    try:
        # search in original files
        cmd = f'grep "{row.glibc} (" /opt/output/temp/*.c.004t.original'
        ret = subprocess.check_output(cmd, shell=True).decode().split("\n")
        ret.pop(-1)
        ret = list(map(lambda x: x.split(":")[0].replace("/opt/output/temp/","").replace(".c.004t.original",""), ret))
        sorted_glibc_syscall_df["originals"][index].extend(ret)
    except:
        print(f"no {row.glibc} in original")
    try:
        # search in test files
        cmd = f'find /opt/output/temp/testcase/result/{row.glibc}-*.txt'
        ret = subprocess.check_output(cmd, shell=True).decode().split("\n")
        ret.pop(-1)
        ret = list(map(lambda x: x.replace("/opt/output/temp/testcase/result/","").replace(".txt",""), ret))
        sorted_glibc_syscall_df["test_result"][index].extend(ret)
    except:
        print(f"no {row.glibc} in test code result")
        
for index, row in sorted_glibc_syscall_df.iterrows():
    sorted_glibc_syscall_df["codes"][index] = set(sorted_glibc_syscall_df["codes"][index])
    sorted_glibc_syscall_df["originals"][index] = set(sorted_glibc_syscall_df["originals"][index])
    sorted_glibc_syscall_df["test_result"][index] = set(sorted_glibc_syscall_df["test_result"][index])

no getaddrinfo in c codes
no getaddrinfo in original
no gethostbyname in c codes
no gethostbyname in original
no clnt_create in c codes
no clnt_create in original
no getnameinfo in c codes
no getnameinfo in original
no forkpty in c codes
no forkpty in original
no aio_write in original
no aio_read in original
no daemon in original
no aio_fsync in original
no authunix_create_default in c codes
no authunix_create_default in original
no dprintf in original
no fputs in c codes
no fputs in original
no putchar in original
no puts in original
no vfprintf in original
no __getdelim in c codes
no __getdelim in original
no getc_unlocked in c codes
no getc_unlocked in original
no errx in original
no keyctl_set_timeout in c codes
no keyctl_set_timeout in original
no gethostname in c codes
no gethostname in original
no __fxstat in c codes
no __fxstat in original
no __xstat in c codes
no __xstat in original
no mmap64 in c codes
no mmap64 in original
no __fxstatat in c codes
no __fxstatat in original
n

/tmp/ipykernel_1131056/3138960524.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sorted_glibc_syscall_df["codes"][index] = set(sorted_glibc_syscall_df["codes"][index])
/tmp/ipykernel_1131056/3138960524.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sorted_glibc_syscall_df["originals"][index] = set(sorted_glibc_syscall_df["originals"][index])
/tmp/ipykernel_1131056/3138960524.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

In [132]:
sorted_glibc_syscall_df.head()

,glibc,syscall_num,syscall_sequence,codes,originals,test_result
0,initgroups,189.0,"[257, 0, 3, 12, 12, 41, 42, 3, 41, 42, 3, 257,...",{3},{3},"{initgroups-default-default, initgroups-3-prep..."
1,getpwuid,33.0,"[12, 12, 41, 42, 3, 41, 42, 3, 257, 5, 0, 0, 3...","{47163, 50541}","{47163, 50541}",{getpwuid-default-default}
2,pclose,23.0,"[61, 13, 59, 9, 56, 10, 17, 12, 11, 3, 257, 5,...","{18411, 50541, 35161, 47163, 42275, 15944}","{18411, 50541, 35161, 47163, 42275, 15944}",{pclose-default-default}
3,getaddrinfo,22.0,"[9, 257, 3, 0, 10, 5, 11, 41, 42, 307, 47, 7, ...",{},{},{getaddrinfo-default-default}
4,gethostbyname,18.0,"[9, 10, 257, 3, 0, 5, 11, 42, 44, 41, 12, 4, 7...",{},{},{gethostbyname-default-default}


In [133]:
sorted_glibc_syscall_df.to_excel("glibc_verification.xlsx")

# 전체 N-gram 그래프의 Break Down 그래프
### 여기서는Token Size가 9인 것을 예로 들어 설명

- 전체 9-gram의 개수 :4개
- 9-gram의 전체 익스플로잇 캡쳐 횟수: 26

In [161]:
# 9-gram인 패턴만 뽑기
nine_gram_df = soft_dedup_ngramdf.copy()
nine_gram_df.head()

,N,Ngram_syscall,num_of_exploit,EIDs
0,NaN,"[14, 248, 248, 248, 41, 248, 248, 248, 54, 0, ...",2,"[40871, 47170]"
156,NaN,"[230, 0, 3, 5, 9, 10, 12, 13, 14, 17, 56, 218,...",2,"[40871, 47170]"
251,NaN,"[12, 12, 5, 0, 257, 4, 12, 12, 5, 0, 257, 4, 1...",2,"[47163, 50541]"
269,NaN,"[102, 104, 272, 272, 257, 1, 3, 257, 1, 3, 257...",2,"[43418, 47169]"
275,NaN,"[12, 12, 257, 12, 12, 257, 5, 0, 5, 0, 5, 0, 5...",2,"[14814, 15023]"


In [156]:
import numpy as np
for index, row in nine_gram_df.iterrows():
    if not row.N == 9:
        nine_gram_df["N"][index] = np.nan
    
nine_gram_df = nine_gram_df.dropna(axis=0)
nine_gram_df.head()

/tmp/ipykernel_1131056/845572546.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nine_gram_df["N"][index] = np.nan


,N,Ngram_syscall,num_of_exploit,EIDs
12581,9,"[257, 4, 230, 257, 4, 230, 257, 4, 230]",3,"[2006, 38390, 39230]"
12591,9,"[12, 12, 257, 12, 12, 257, 5, 0, 5]",7,"[14814, 15023, 15285, 15704, 15774, 15916, 40839]"
12763,9,"[5, 0, 5, 0, 5, 0, 5, 0, 3]",18,"[14814, 15023, 15285, 15704, 15774, 15916, 177..."
12794,9,"[12, 12, 257, 5, 0, 5, 0, 5, 0]",22,"[14814, 15023, 15024, 15285, 15704, 15774, 159..."


In [158]:
# sysnum -> sysname
ngram_syscall = list()
for index, row in nine_gram_df.iterrows():
    for syscall in row.Ngram_syscall:
        ngram_syscall.append(convert_num_name(syscall))
    nine_gram_df["Ngram_syscall"][index] = ngram_syscall
    ngram_syscall = []

# 엑셀에 저장
nine_gram_df.to_excel("breakdown_9-gram.xlsx")

/tmp/ipykernel_1131056/3127638939.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nine_gram_df["Ngram_syscall"][index] = ngram_syscall


# 카테고리별 패턴 개수 / 익스플로잇 수
## 전체 익스플로잇 대상 패턴 개수

In [137]:
import pandas as pd

allExploitCatInfo = pd.read_pickle('onlyCategoryWithEID.pkl')
allExploitCatInfo.head()

,EID,Kernel_Component,Vulnerability_Type
0,2013,fs,BA
1,4756,net,BA
2,8678,kernel,RC
3,10613,fs,ETC
4,15023,arch,BA


In [138]:
# EID에 해당하는 패턴 열 추가

# pattern 열 만들기
pattern = [[] for i in range(len(allExploitCatInfo.EID.to_list()))]
allExploitCatInfo["pattern"] = pattern
allExploitCatInfo.head()

# pattern 열에 데이터 채워넣기
for index1, row1 in soft_dedup_ngramdf.iterrows():
    for eid in row1.EIDs:
        eid = int(eid)
        for index2, row2 in allExploitCatInfo.iterrows():
            if row2.EID == eid:
                allExploitCatInfo["pattern"][index2].append(row1.Ngram_syscall)
allExploitCatInfo.head()

,EID,Kernel_Component,Vulnerability_Type,pattern
0,2013,fs,BA,"[[12, 12, 12, 12], [257, 5, 9], [12, 12, 1], [..."
1,4756,net,BA,"[[12, 12, 56, 56, 56], [41, 107, 59], [12, 12,..."
2,8678,kernel,RC,"[[101, 101, 101, 101, 101, 101, 230], [101, 10..."
3,10613,fs,ETC,"[[102, 104, 117, 119, 9, 9, 9, 9, 11, 22, 3, 1..."
4,15023,arch,BA,"[[12, 12, 257, 12, 12, 257, 5, 0, 5, 0, 5, 0, ..."


In [139]:
kernel_component_set = list(set(allExploitCatInfo.Kernel_Component.to_list()))
vulnerability_set = list(set(allExploitCatInfo.Vulnerability_Type.to_list()))

# Initialization (eid Category)
category_eid = pd.DataFrame(columns = kernel_component_set, index=vulnerability_set)
for index, row in category_eid.iterrows():
    for c in kernel_component_set:
        category_eid[c][index] = []
category_eid_cnt = pd.DataFrame(columns = kernel_component_set, index=vulnerability_set)
for index, row in category_eid_cnt.iterrows():
    for c in kernel_component_set:
        category_eid_cnt[c][index] = []
        
# fill content
for index, row in allExploitCatInfo.iterrows():
    category_eid[row.Kernel_Component][row.Vulnerability_Type].append(row.EID)

for index, row in category_eid.iterrows():
    for c in kernel_component_set:
        category_eid_cnt[c][index] = len(set(category_eid[c][index]))
category_eid_cnt

,net,arch,fs,ipc,lib,security,kernel,drivers,mm,include
RC,4,0,6,0,0,0,5,1,2,1
BA,14,3,2,0,2,0,0,2,4,1
IV,10,1,3,0,0,0,3,0,0,0
AC,4,4,12,0,0,0,12,1,0,1
ETC,0,0,1,0,0,0,0,0,0,0
IL,1,0,0,1,0,0,2,0,1,0
IC,3,0,6,0,0,2,5,1,0,0
PU,6,0,6,1,0,0,1,0,1,0


In [140]:
# Initialization (eid Pattern)
category_pattern = pd.DataFrame(columns = kernel_component_set, index=vulnerability_set)
for index, row in category_pattern.iterrows():
    for c in kernel_component_set:
        category_pattern[c][index] = []
category_pattern_cnt = pd.DataFrame(columns = kernel_component_set, index=vulnerability_set)
for index, row in category_pattern_cnt.iterrows():
    for c in kernel_component_set:
        category_pattern_cnt[c][index] = []
        
# fill contents
for index, row in allExploitCatInfo.iterrows():
    category_pattern[row.Kernel_Component][row.Vulnerability_Type].extend(row.pattern)

for index, row in category_pattern.iterrows():
    for c in kernel_component_set:
        category_pattern_cnt[c][index] = len(category_pattern[c][index])
category_pattern_cnt

,net,arch,fs,ipc,lib,security,kernel,drivers,mm,include
RC,264,0,114,0,0,0,76,46,58,0
BA,453,65,28,0,70,0,0,17,128,35
IV,159,4,100,0,0,0,87,0,0,0
AC,144,22,266,0,0,0,420,94,0,4
ETC,0,0,15,0,0,0,0,0,0,0
IL,28,0,0,37,0,0,12,0,48,0
IC,164,0,94,0,0,36,34,41,0,0
PU,258,0,90,37,0,0,39,0,48,0


In [141]:
# excel로 저장
category_pattern_cnt.to_excel("attack_category_pattern.xlsx")

## 같은 카테고리 대상 패턴 개수

In [142]:
def combine_ngram_cnt(combine_EID):
    vulnerability_ngram = dict()
    
    for index, row in soft_dedup_ngramdf.iterrows():
        for EID in combine_EID:
            if str(EID) in row.EIDs:
                if ",".join(row.Ngram_syscall) in vulnerability_ngram.keys():
                    vulnerability_ngram[",".join(row.Ngram_syscall)] += 1
                else:
                    vulnerability_ngram[",".join(row.Ngram_syscall)] = 1

    # sysnum -> sysname
    vulnerability_ngram_sysname = dict()
    for ngram, cnt in vulnerability_ngram.items():
        sys_name_list = list()
        for syscall in ngram.split(","):
            sys_name_list.append(convert_num_name(syscall))
        vulnerability_ngram_sysname[",".join(sys_name_list)] = vulnerability_ngram[ngram]
    
    # 중복 제거
    sorted_vulnerability_ngram_sysname = sorted(vulnerability_ngram_sysname.items(), key=lambda x: x[1], reverse=True)
    sorted_vulnerability_ngram_sysname = list(map(lambda x: list(x), sorted_vulnerability_ngram_sysname))
    for i, ngram1 in enumerate(sorted_vulnerability_ngram_sysname):
        if ngram1 == "": continue
        for j, ngram2 in enumerate(sorted_vulnerability_ngram_sysname[i+1:]):
            if ngram2 == "": continue
            if ngram2[0] in ngram1[0]:
                sorted_vulnerability_ngram_sysname[i+j+1][0] = ""
    
    final_vulnerability_ngram = list()
    for ngram in sorted_vulnerability_ngram_sysname:
        if not ngram[0] == "":
            final_vulnerability_ngram.append(ngram)
            
    final_vulnerability_ngram_df = pd.DataFrame(columns=["N","ngram","percent","cnt"])
    row = {"ngram":"", "percent":0, "N":0,"cnt":0}
    for ngram in final_vulnerability_ngram:
        row["ngram"] = ngram[0]
        row["N"] = len(ngram[0].split(","))
        row["cnt"] = ngram[1]
        row["percent"] = ngram[1]/len(combine_EID)*100
        final_vulnerability_ngram_df = final_vulnerability_ngram_df.append(row, ignore_index=True)
        row = {"ngram":"", "percent":0, "N":0,"cnt":0}
        
    print(len(final_vulnerability_ngram_df))
    return len(final_vulnerability_ngram_df)
#     final_vulnerability_ngram_df.to_excel(f"combination_{combine_name}_ngram.xlsx")
#     print(f"Finished making {combine_name} ngram excel! - total EID #: {len(combine_EID)}")

In [143]:
# Initialization (Category Pattern)
pattern_in_category_cnt = pd.DataFrame(columns = kernel_component_set, index=vulnerability_set)
for index, row in pattern_in_category_cnt.iterrows():
    for c in kernel_component_set:
        pattern_in_category_cnt[c][index] = []
        
        
for kc in kernel_component_set:
    for v in vulnerability_set:
        if category_eid[kc][v]:
            pattern_in_category_cnt[kc][v] = combine_ngram_cnt(list(set(category_eid[kc][v])))
pattern_in_category_cnt

41
92
38
35
10
34
66
16
4
8
38
11
18
92
2
25
32
13
19
10
5
17
31
131
4
15
8
15
9
8
18
20
37
10
10
0
14
1


,net,arch,fs,ipc,lib,security,kernel,drivers,mm,include
RC,41,[],38,[],[],[],17,15,20,0
BA,92,16,11,[],10,[],[],9,37,14
IV,38,4,18,[],[],[],31,[],[],[]
AC,35,8,92,[],[],[],131,8,[],1
ETC,[],[],2,[],[],[],[],[],[],[]
IL,10,[],[],13,[],[],4,[],10,[]
IC,34,[],25,[],[],5,15,18,[],[]
PU,66,[],32,19,[],[],8,[],10,[]


In [144]:
pattern_in_category_cnt.to_excel("attack_category_inter_pattern.xlsx")

## 각 카테고리 별 N-gram 개수

### 취약점 유형 별 N-gram 개수

In [145]:
def vulnerability_ngram_cnt(vulnerability_EID):
    vulnerability_ngram = dict()
    
    for index, row in soft_dedup_ngramdf.iterrows():
        for EID in vulnerability_EID:
            if str(EID) in row.EIDs:
                if ",".join(row.Ngram_syscall) in vulnerability_ngram.keys():
                    vulnerability_ngram[",".join(row.Ngram_syscall)] += 1
                else:
                    vulnerability_ngram[",".join(row.Ngram_syscall)] = 1

    # sysnum -> sysname
    vulnerability_ngram_sysname = dict()
    for ngram, cnt in vulnerability_ngram.items():
        sys_name_list = list()
        for syscall in ngram.split(","):
            sys_name_list.append(convert_num_name(syscall))
        vulnerability_ngram_sysname[",".join(sys_name_list)] = vulnerability_ngram[ngram]
    
    # 중복 제거
    sorted_vulnerability_ngram_sysname = sorted(vulnerability_ngram_sysname.items(), key=lambda x: x[1], reverse=True)
    sorted_vulnerability_ngram_sysname = list(map(lambda x: list(x), sorted_vulnerability_ngram_sysname))
    for i, ngram1 in enumerate(sorted_vulnerability_ngram_sysname):
        if ngram1 == "": continue
        for j, ngram2 in enumerate(sorted_vulnerability_ngram_sysname[i+1:]):
            if ngram2 == "": continue
            if ngram2[0] in ngram1[0]:
                sorted_vulnerability_ngram_sysname[i+j+1][0] = ""
    
    final_vulnerability_ngram = list()
    for ngram in sorted_vulnerability_ngram_sysname:
        if not ngram[0] == "":
            final_vulnerability_ngram.append(ngram)
            
    final_vulnerability_ngram_df = pd.DataFrame(columns=["N","ngram","percent","cnt"])
    row = {"ngram":"", "percent":0, "N":0,"cnt":0}
    for ngram in final_vulnerability_ngram:
        row["ngram"] = ngram[0]
        row["N"] = len(ngram[0].split(","))
        row["cnt"] = ngram[1]
        row["percent"] = ngram[1]/len(vulnerability_EID)*100
        final_vulnerability_ngram_df = final_vulnerability_ngram_df.append(row, ignore_index=True)
        row = {"ngram":"", "percent":0, "N":0,"cnt":0}
    
    print(len(final_vulnerability_ngram_df))
    return len(final_vulnerability_ngram_df)

In [146]:
# Initialization (Vulnerability Pattern)
pattern_in_vulnerability_cnt = pd.DataFrame(index=vulnerability_set)
pattern_in_vulnerability_cnt["pattern_cnt"] = [[] for i in range(len(vulnerability_set))]
        
for v in vulnerability_set:
    if v == "ETC":
        continue
    print(v)
    v_category_eid = category_eid.loc[[v],:]
    all_v_eid = list()
    for kc in kernel_component_set:
        if v_category_eid[[kc][0]].to_list()[0]:
            all_v_eid.extend(v_category_eid[[kc][0]].to_list()[0])
    print(all_v_eid)
    pattern_in_vulnerability_cnt["pattern_cnt"][v] = vulnerability_ngram_cnt(list(set(all_v_eid)))
    all_v_eid = []
pattern_in_vulnerability_cnt

RC
[40871, 43418, 47169, 47170, 44302, 895, 33321, 33322, 40812, 43345, 8678, 20720, 20721, 8673, 34134, 33516, 40616, 40839, 32751]
134
BA
[4756, 40049, 44300, 17787, 41995, 50135, 9479, 9545, 15916, 15944, 41994, 43418, 47168, 47169, 15023, 30604, 4460, 2013, 42887, 42275, 42276, 37167, 37183, 42275, 42276, 40616, 40839, 15024]
172
IV
[15285, 24555, 24746, 25288, 25289, 31347, 33336, 44299, 8572, 926, 926, 31574, 20979, 5092, 5092, 5093, 5093, 43029, 43127, 34134]
79
AC
[15704, 17787, 15916, 15944, 27297, 37722, 30604, 4460, 18411, 2004, 2005, 2006, 2031, 34923, 35161, 37292, 39166, 39230, 6851, 50808, 50808, 38390, 12, 20000, 22362, 22363, 25450, 3, 33824, 35370, 47163, 50541, 15704, 15774, 15774, 718]
233
IL
[17391, 17787, 44298, 45010, 44303]
36
IC
[14814, 41994, 47168, 1397, 19419, 25202, 45516, 5092, 5093, 39277, 40003, 25444, 26131, 33589, 44298, 45010, 9083]
91
PU
[15704, 41458, 9542, 9575, 40871, 47170, 33523, 50808, 33321, 33322, 40812, 43345, 50808, 45553, 15704, 44303]
124

,pattern_cnt
RC,134
BA,172
IV,79
AC,233
ETC,[]
IL,36
IC,91
PU,124


### 커널 컴포넌트 별 N-gram 개수

In [147]:
def kernel_component_ngram_cnt(kernel_component_EID):
    kernel_component_ngram = dict()
    
    for index, row in soft_dedup_ngramdf.iterrows():
        for EID in kernel_component_EID:
            if str(EID) in row.EIDs:
                if ",".join(row.Ngram_syscall) in kernel_component_ngram.keys():
                    kernel_component_ngram[",".join(row.Ngram_syscall)] += 1
                else:
                    kernel_component_ngram[",".join(row.Ngram_syscall)] = 1
    
    # sysnum -> sysname
    kernel_component_ngram_sysname = dict()
    for ngram, cnt in kernel_component_ngram.items():
        sys_name_list = list()
        for syscall in ngram.split(","):
            sys_name_list.append(convert_num_name(syscall))
        kernel_component_ngram_sysname[",".join(sys_name_list)] = kernel_component_ngram[ngram]
    
    # 중복 제거
    sorted_kernel_component_ngram_sysname = sorted(kernel_component_ngram_sysname.items(), key=lambda x: x[1], reverse=True)
    sorted_kernel_component_ngram_sysname = list(map(lambda x: list(x), sorted_kernel_component_ngram_sysname))
    for i, ngram1 in enumerate(sorted_kernel_component_ngram_sysname):
        if ngram1 == "": continue
        for j, ngram2 in enumerate(sorted_kernel_component_ngram_sysname[i+1:]):
            if ngram2 == "": continue
            if ngram2[0] in ngram1[0]:
                sorted_kernel_component_ngram_sysname[i+j+1][0] = ""
    
    final_kernel_component_ngram = list()
    for ngram in sorted_kernel_component_ngram_sysname:
        if not ngram[0] == "":
            final_kernel_component_ngram.append(ngram)
            
    final_kernel_component_ngram_df = pd.DataFrame(columns=["N","ngram","percent"])
    row = {"ngram":"", "percent":0, "N":0, "cnt":0}
    for ngram in final_kernel_component_ngram:
        row["ngram"] = ngram[0]
        row["N"] = len(ngram[0].split(","))
        row["cnt"] =  ngram[1]
        row["percent"] = ngram[1]/len(kernel_component_EID)*100
        final_kernel_component_ngram_df = final_kernel_component_ngram_df.append(row, ignore_index=True)
        row = {"ngram":"", "percent":0, "N":0, "cnt":0}
    
#     final_kernel_component_ngram_df.to_excel(f"kernel_component_{kernel_component_name}_ngram.xlsx")
#     print(f"Finished making {kernel_component_name} ngram excel! - total EID #: {len(kernel_component_EID)}")
    print("pattern # : ",len(final_kernel_component_ngram_df))
    return len(final_kernel_component_ngram_df)

In [148]:
# Initialization (Vulnerability Pattern)
pattern_in_kernel_component_cnt = pd.DataFrame(index=kernel_component_set)
pattern_in_kernel_component_cnt["pattern_cnt"] = [[] for i in range(len(kernel_component_set))]
        
for kc in kernel_component_set:
    print(kc)
    kc_category_eid = category_eid[[kc]]
    all_kc_eid = list()
    for index, row in kc_category_eid.iterrows():
        all_kc_eid.extend(row.to_list()[0])
    print("exploit # : ", len(set(all_kc_eid)))
    pattern_in_kernel_component_cnt["pattern_cnt"][kc] = kernel_component_ngram_cnt(list(set(all_kc_eid)))
    all_kc_eid = []
pattern_in_kernel_component_cnt

net
exploit # :  32
pattern # :  177
arch
exploit # :  6
pattern # :  26
fs
exploit # :  29
pattern # :  167
ipc
exploit # :  2
pattern # :  33
lib
exploit # :  2
pattern # :  10
security
exploit # :  2
pattern # :  5
kernel
exploit # :  24
pattern # :  173
drivers
exploit # :  5
pattern # :  46
mm
exploit # :  5
pattern # :  48
include
exploit # :  3
pattern # :  15


,pattern_cnt
net,177
arch,26
fs,167
ipc,33
lib,10
security,5
kernel,173
drivers,46
mm,48
include,15


# 유명한 Container Escape 익스플로잇 패턴 조사

In [82]:
famous_ngram_df = soft_dedup_ngramdf.copy()
famous_ngram_df.head()

,N,Ngram_syscall,num_of_exploit,EIDs
0,66,"[14, 248, 248, 248, 41, 248, 248, 248, 54, 0, ...",2,"[40871, 47170]"
156,49,"[230, 0, 3, 5, 9, 10, 12, 13, 14, 17, 56, 218,...",2,"[40871, 47170]"
251,45,"[12, 12, 5, 0, 257, 4, 12, 12, 5, 0, 257, 4, 1...",2,"[47163, 50541]"
269,44,"[102, 104, 272, 272, 257, 1, 3, 257, 1, 3, 257...",2,"[43418, 47169]"
275,44,"[12, 12, 257, 12, 12, 257, 5, 0, 5, 0, 5, 0, 5...",2,"[14814, 15023]"


### 유명 익스플로잇 패턴 모으는 함수

In [80]:
# 해당 익스플로잇 코드가 가진 패턴 모으기 
def first_all_pattern_in_famous_exploit(eids, vulname):
    # Initialization
    famous_vul_ngram_df = pd.DataFrame(columns=["EID","pattern","co_captured_EIDs"])
    famous_vul_dict = {"EID":"", "pattern":[], "co_captured_EIDs":[]}

    for eid in eids:
        famous_vul_dict = {"EID":eid, "pattern":[], "co_captured_EIDs":[]}
        for index2, row2 in famous_ngram_df.iterrows():
            famous_vul_dict = {"EID":eid, "pattern":[], "co_captured_EIDs":[]}
            if eid in row2.EIDs:
                # pattern
                pattern_sysname = []
                for sysnum in row2.Ngram_syscall:
                    pattern_sysname.append(convert_num_name(sysnum))
                famous_vul_dict["pattern"] = pattern_sysname
                # co-captured EIDs
                captured_eid = row2.EIDs.copy()
                captured_eid.remove(eid)
                famous_vul_dict["co_captured_EIDs"] = captured_eid
                # append dict to df
                famous_vul_ngram_df = famous_vul_ngram_df.append(famous_vul_dict, ignore_index=True)

    famous_vul_ngram_df.to_excel(f"famous_{vulname}_ngram.xlsx")
    return famous_vul_ngram_df

In [214]:
# 해당 익스플로잇 패턴의 총 합계 구하기
def second_total_pattern_in_famous_exploit(famous_vul_ngram_df, vulname):
    famous_vul_ngram_total_df = pd.DataFrame(columns=["EID","total_pattern","num_of_total_pattern","total_co_captured_EIDs", "num_of_total_co_captured_EIDs"])
    famous_vul_ngram_total_dict = {"EID":"","total_pattern":[],"num_of_total_pattern":0,"total_co_captured_EIDs":[],"num_of_total_co_captured_EIDs":0}

    for index, row in famous_vul_ngram_df.iterrows():
        if famous_vul_ngram_total_dict["EID"]=="":
            famous_vul_ngram_total_dict["EID"] = row.EID
            famous_vul_ngram_total_dict["total_pattern"].append(row.pattern)
            famous_vul_ngram_total_dict["total_co_captured_EIDs"].extend(row.co_captured_EIDs)
        elif famous_vul_ngram_total_dict["EID"] == row.EID:
            famous_vul_ngram_total_dict["total_pattern"].append(row.pattern)
            famous_vul_ngram_total_dict["total_co_captured_EIDs"].extend(row.co_captured_EIDs)
        else:
            # append
            famous_vul_ngram_total_dict["num_of_total_pattern"] = len(famous_vul_ngram_total_dict["total_pattern"])
            famous_vul_ngram_total_dict["num_of_total_co_captured_EIDs"] = len(set(famous_vul_ngram_total_dict["total_co_captured_EIDs"]))
            famous_vul_ngram_total_df = famous_vul_ngram_total_df.append(famous_vul_ngram_total_dict, ignore_index=True)
            # initialization
            famous_vul_ngram_total_dict = {"EID":"","total_pattern":[],"num_of_total_pattern":0,"total_co_captured_EIDs":[],"num_of_total_co_captured_EIDs":0}
            # new EID
            famous_vul_ngram_total_dict["EID"] = row.EID
    # append
    famous_vul_ngram_total_dict["num_of_total_pattern"] = len(famous_vul_ngram_total_dict["total_pattern"])
    famous_vul_ngram_total_dict["num_of_total_co_captured_EIDs"] = len(set(famous_vul_ngram_total_dict["total_co_captured_EIDs"]))
    famous_vul_ngram_total_df = famous_vul_ngram_total_df.append(famous_vul_ngram_total_dict, ignore_index=True)

    # excel에 저장
    famous_vul_ngram_total_df.to_excel(f"famous_{vulname}_ngram_total.xlsx")
    return famous_vul_ngram_total_df

#### DirtyCOW (CVE-2016-5195)
- Exploit: 40616, 40839

In [215]:
# 해당 익스플로잇 코드가 가진 패턴 모으기 
dirtycow_eids = ["40616", "40839"]
dirtycow_ngram_df = first_all_pattern_in_famous_exploit(dirtycow_eids, "dirtycow")
dirtycow_ngram_df.head()

,EID,pattern,co_captured_EIDs
0,40616,"[mmap, read, close, fstat, mmap, mprotect, brk...",[43127]
1,40616,"[read, close, fstat, mmap, mprotect, brk, rt_s...","[43127, 44302]"
2,40616,"[read, close, fstat, mmap, mprotect, brk, rt_s...","[40871, 43127, 43345, 44302, 47170]"
3,40616,"[mmap, read, close, fstat, mmap, mprotect, brk...","[35370, 41995, 43127]"
4,40616,"[read, close, fstat, mmap, mprotect, brk, rt_s...","[33516, 35370, 40839, 40871, 41995, 43127, 433..."


In [216]:
# 해당 익스플로잇 패턴의 총 합계 구하기
dirtycow_ngram_total_df = second_total_pattern_in_famous_exploit(dirtycow_ngram_df, "dirtycow")
dirtycow_ngram_total_df

,EID,total_pattern,num_of_total_pattern,total_co_captured_EIDs,num_of_total_co_captured_EIDs
0,40616,"[[mmap, read, close, fstat, mmap, mprotect, br...",27,"[43127, 43127, 44302, 40871, 43127, 43345, 443...",82
1,40839,"[[brk, brk, openat, brk, brk, openat, fstat, r...",30,"[14814, 15023, 15285, 15704, 15774, 15916, 148...",83


#### DirtyPipe (CVE-2022-0847)
- Exploit: 50808

In [217]:
# 해당 익스플로잇 코드가 가진 패턴 모으기 
dirtypipe_eids = ["50808"]

dirtypipe_ngram_df = first_all_pattern_in_famous_exploit(dirtypipe_eids, "dirtypipe")
dirtypipe_ngram_df.head()

,EID,pattern,co_captured_EIDs
0,50808,"[rt_sigaction, rt_sigaction, rt_sigprocmask, m...","[1397, 14814, 2004, 2006, 2031, 24746, 3, 3134..."
1,50808,"[openat, brk, brk]","[14814, 15023, 15285, 15704, 15774, 15916, 209..."
2,50808,"[brk, brk, lseek]",[]
3,50808,"[write, write, write]","[33516, 37167, 45516, 895]"
4,50808,"[lseek, read, close]",[3]


In [218]:
# 해당 익스플로잇 패턴의 총 합계 구하기
dirtypipe_ngram_total_df = second_total_pattern_in_famous_exploit(dirtypipe_ngram_df, "dirtypipe")
dirtypipe_ngram_total_df

,EID,total_pattern,num_of_total_pattern,total_co_captured_EIDs,num_of_total_co_captured_EIDs
0,50808,"[[rt_sigaction, rt_sigaction, rt_sigprocmask, ...",18,"[1397, 14814, 2004, 2006, 2031, 24746, 3, 3134...",82


#### heap OOB write (CVE-2021-22555)
- Exploit: 50135

In [83]:
# 해당 익스플로잇 코드가 가진 패턴 모으기 
heapoob_eids = ["50135"]
heapoob_ngram_df = first_all_pattern_in_famous_exploit(heapoob_eids, "heapoob")
heapoob_ngram_df.head()

,EID,pattern,co_captured_EIDs
0,50135,"[socket, socketpair, socketpair, socketpair]",[41458]
1,50135,"[unshare, unshare]","[40871, 41458, 41994, 43127, 43418, 47168, 471..."


In [219]:
# 해당 익스플로잇 패턴의 총 합계 구하기
heapoob_ngram_total_df = second_total_pattern_in_famous_exploit(heapoob_ngram_df, "heapoob")
heapoob_ngram_total_df

,EID,total_pattern,num_of_total_pattern,total_co_captured_EIDs,num_of_total_co_captured_EIDs
0,50135,"[[socket, socketpair, socketpair, socketpair],...",2,"[41458, 40871, 41458, 41994, 43127, 43418, 471...",8


#### Unknow Name, packet_set_ring (CVE-2017-7308)
- Exploit: 41994, 47168

In [221]:
# 해당 익스플로잇 코드가 가진 패턴 모으기 
packetsetring_eids = ["41994","47168"]
packetsetring_ngram_df = first_all_pattern_in_famous_exploit(packetsetring_eids, "packetsetring")
packetsetring_ngram_df.head()

,EID,pattern,co_captured_EIDs
0,41994,"[socket, setsockopt, setsockopt, access, acces...",[47168]
1,41994,"[getuid, getgid, unshare, unshare, openat, wri...","[41458, 43418, 47168, 47169]"
2,41994,"[socket, setsockopt, setsockopt, access, acces...",[47168]
3,41994,"[unshare, unshare, openat, write, close, opena...","[41458, 43127, 43418, 47168, 47169]"
4,41994,"[rt_sigaction, rt_sigaction, rt_sigprocmask, m...","[1397, 14814, 2004, 2006, 2031, 24746, 3, 3134..."


In [222]:
# 해당 익스플로잇 패턴의 총 합계 구하기
packetsetring_ngram_total_df = second_total_pattern_in_famous_exploit(packetsetring_ngram_df, "packetsetring")
packetsetring_ngram_total_df

,EID,total_pattern,num_of_total_pattern,total_co_captured_EIDs,num_of_total_co_captured_EIDs
0,41994,"[[socket, setsockopt, setsockopt, access, acce...",33,"[47168, 41458, 43418, 47168, 47169, 47168, 414...",76
1,47168,"[[brk, brk, openat, fstat, read, fstat, read, ...",72,"[47169, 47170, 41458, 41994, 43418, 47169, 419...",89
